In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                 # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../")            #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
#using MetaGraphs
using MetaGraphsNext
using GraphPlot
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff

# Tools

## Enumerator

Tools to compute a running enumeration of a "list" of objects.

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

## Running Statistics

Tools to compute running statistics such as mean and std from a stream of numbers.

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

In [ ]:
list_v = 10 .+ 5.0*randn(1000)
;

In [ ]:
rs = RunningStat()

In [ ]:
for v in list_v
    push!(rs,v)
end

In [ ]:
mean(rs),std(rs)

In [ ]:
mean(list_v),std(list_v)

## Coefficient of determiantion, $R^2$

Tools to compute the $R^2$ coefficient to evaluate the fit of functions.

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

In [ ]:
# Define the model function
@. model(x, p) = p[1] * x + p[2]

# Sample data (example)
x_data = [1.0, 2.0, 3.0, 4.0, 5.0]
y_data = [2.2, 2.8, 3.6, 4.5, 5.1]

# Initial guess for parameters
p0 = [1.0, 1.0]

# Perform curve fitting
fit = LsqFit.curve_fit(model, x_data, y_data, p0)

# Compute fitted values
y_fit = model(x_data, coef(fit))

# Compute R_sqr
R_sqr = R_squared(y_data,y_fit)

In [ ]:
plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"x")
plot!(ylabel=L"y")
scatter!(x_data,y_data,label="data")
plot!(x_data,y_fit,label="fit, \$R^2=0.995\$")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
#plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(
#    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
#    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
#    tickfont=font(11),
#)

## Sigmoid functions

The logistic sigmoid function is

$$
f(x) = \frac{A}{1+e^{-B(x-x_0)}}
$$

Its derivative is

$$
f'(x) = \frac{A\,B\,e^{-B(x-x_0)}}{(1+e^{-B(x-x_0)})^2}
$$

and its primitive is

$$
F(x) = \int dx\,f(x) = \frac{A}{B}\ln(1+e^{B(x-x_0)})+C
$$


In [ ]:
x0 = 1.0
A = 1.0
B = 3.0
C = 0.0
p0 = [x0,A,B,C] # p1=x0, p2=A, p3=B, p4=C
@. sigmoid(x,p)  = p[2]/(1+exp(-p[3]*(x-p[1]))) # sigmoid
@. dsigmoid(x,p) = p[2]*p[3]*exp(-p[3]*(x-p[1]))/(1+exp(-p[3]*(x-p[1])))^2 # derivative of sigmoid
@. psigmoid(x,p) = p[2]/p[3]*log(1+exp(p[3]*(x-p[1])))+p[4] # primitive of sigmoid

### Test

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"f(x)",
)
plot!(-2:0.01:4.0,x->sigmoid(x,p0),label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"df/dx",    
)
plot!(-2:0.01:4.0,x->dsigmoid(x,p0),label="")
scatter!([x0],[A*B/4],label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"F(x)",
)
plot!(-2:0.01:4.0,x->psigmoid(x,p0),label="")

## Log-range

Tools to create a range of numbers but in logarithmic scale.

In [ ]:
function logrange(start,stepmul,stop)
    @assert start>0
    @assert stepmul>1
    r = []
    s = start
    while s <= stop
        push!(r,s)
        s *= stepmul
    end
    return r
end

### Test

In [ ]:
logrange(1e-4,1.1,1)

# Generate networks

In [ ]:
function main_component(g)
    c = connected_components(g)
    _,i = findmax(length.(c))
    return g[c[i]]
end

In [ ]:
sorted_connected_components(g) = sort(connected_components(g), by = c -> length(c),rev=true)

In [ ]:
"""
Generates a 1D lattice Graph of N nodes and k=2z nearest neighbors.
"""
function lattice_1D_to_g(N,z=1)
    g = Graph(N)
    for i in 1:N
        for j in 1:z
            if i+j <= N
                add_edge!(g,i,i+j)            
            end
        end
    end
    return g
end

## Tests

In [ ]:
sorted_connected_components(g)

# Load networks

In [ ]:
@load "../redes/redes_dict.jld2" redes_jld2

In [ ]:
redes_jld2

In [ ]:
countries = Dict()
metrics = Dict()
for (c,m) in keys(redes_jld2)
    #@show c,m
    enum!(countries,c)
    enum!(metrics,m)
end

In [ ]:
countries

In [ ]:
metrics

In [ ]:
redes_wg = Dict()
for (c,m) in keys(redes_jld2)
    vec_i,vec_j,vec_w = redes_jld2[(c,m)]
    N = max(maximum(vec_i),maximum(vec_j))
    g = Graph(N)
    w = zeros(N,N)
    for (i,j,w_ij) in zip(vec_i,vec_j,vec_w)
        add_edge!(g,i,j)
        w[i,j] = w_ij
    end    
    redes_wg[(c,m)] = (g,w)
end

In [ ]:
redes_wg[("Italy", "crossing")]

In [ ]:
@save "../redes/wgraphs_dict.jld2" redes_wg

In [ ]:
@load "../redes/wgraphs_dict.jld2" redes_wg

In [ ]:
redes_wg

# Simplicial Complex

Tools to represent simplicial complexes and associated operators in matrix form. For further details, please read de manuscript.

The array `K[k]` is a representation of $K_k$ and has dimensions $m_k\times(k+1)$ and is used to enumerate the $k$-simplices of the complex.
Here $m_k=|K_k|=\dim C_k=\dim C_k^*$ is the number of $k$-simplices in the simplicial complex.
Especifically, `K[k][l,:]` is an array `[i_0,i_1,...,i_k]` denoting the $l$-th $k$-simplex of the complex.
By assumption, $i_0<i_1<...<i_k$.
The dictionary `I[k]` is the inverse mapping of `K[k]`.
It maps arrays `[i_0,i_1,...,i_k]` to their corresponding index `l` in the enumeration.

## Matricial representation of the (co)-boundary and (co)-differential operators

Recall that, if

$$
\partial_k(e_s) = (\alpha_k)_s{}^r e_r
$$
$$
\partial_k^*(e_s) = (\mu_k)_s{}^r e_r 
$$
$$
d_k(e^s) = (\beta_k)^s{}_r e^r
$$
$$
d_k^*(e^s) = (\mu_k)^s{}_r e^r
$$

and $m_{sr}=\delta_{sr}$, then

$$
(\beta_k)^s{}_r = (\alpha_{k+1})_r{}^s
$$
$$
(\mu_k)_s{}^r = (\alpha_{k})_r{}^s
$$
$$
(\nu_k)^s{}_r = (\beta_{k})^r{}_s = (\alpha_{k+1})_s{}^r
$$

and, therefore

$$
(\partial_k(c))^r = (\alpha_k^T)^r{}_sc^s
$$
$$
(\partial_k^*(c))^r = (\alpha_k)^r{}_sc^s
$$
$$
(d_k(f))_r = (\alpha_{k+1})_r{}^sf_s
$$
$$
(d_k^*(f))_r = (\alpha_{k+1}^T)_r{}^sf_s
$$

Finally, note that $\alpha_k\in \mathbb{R}^{m_{k}\times m_{k-1}}$ for $k\in 1,2,...,\kappa$ and

$$
(\alpha_{k})_{i_0...i_j...i_k}{}^{i_0...\hat{i}_j...i_k} = (-1)^j
$$

Here, $d_0$ and $d_1$ are needed. 
Hence, the matrices $\beta_0=\alpha_1$ and $\beta_1=\alpha_2$ have to be computed, respectively. 
In particular, assuming $i<j<k$, it is found that

$$(\alpha_1)_{ij}{}^{j} = 1$$
$$(\alpha_1)_{ij}{}^{i} = -1$$

and

$$(\alpha_2)_{ijk}{}^{jk} = 1$$
$$(\alpha_2)_{ijk}{}^{ik} = -1$$
$$(\alpha_2)_{ijk}{}^{ij} = 1$$

In [ ]:
struct SimplicialComplex
    K::Any
    I::Any
    alpha::Any
end

In [ ]:
"""
Number of dimensions (kappa+1) of a simplicial complex, k = 0,1,...,kappa.
"""
function ndim(sc::SimplicialComplex)
    return length(sc.K)
end

In [ ]:
"""
Maximum dimension of a simplicial complex.
"""
function kappa(sc::SimplicialComplex)
    return ndim(sc)-1
end

In [ ]:
"""
m_k = dim C_k = dim C_k^*.
"""
function m_k(sc::SimplicialComplex,k::Int)
    if k>=0 && k<=kappa(sc)
        return size(sc.K[k])[1]
    end
    return 0
end

In [ ]:
"""
List of the numbers of non-zero elements in the matricial representations of the (dual)-(co)-boundary operators, alpha_k.
"""
function nnz_alpha(sc::SimplicialComplex)
    return [nnz(sc.alpha[k]) for k in 1:length(sc.alpha)] 
end

In [ ]:
"""
Takes as input a list I of dictionaries I_k mapping simplices [i_0,i_1,...,i_k] to their corresponding enumeration numbers l.
Returns a SimplicialComplex containing:
    K : the list of the inverse mappings K_k of the I_k, and
    I : the list of the mappings I_k passed as input,
    alpha : the list alpha of the corresponding matrices alpha_k.
"""
function I_to_SC(I)
    # Create K
    K = []
    for k in 0:(length(I)-1)
        Kk = zeros(Int64,(length(I[k]),k+1))
        for (s,l) in I[k]
            Kk[l,:] = s
        end
        push!(K,Kk)
    end
    K = OffsetArray(K,0:(length(I)-1))

    # Create A
    alpha = []
    for k in 1:(length(I)-1)
        #@show k
        Kk=K[k]
        Kp=K[k-1] # p = k-1
        Ik=I[k]
        Ip=I[k-1]
        n_k = size(Kk)[1]
        n_p = size(Kp)[1]        
        rows = zeros(Int64,(k+1)*n_k)
        cols = zeros(Int64,(k+1)*n_k)
        vals = zeros(Float64,(k+1)*n_k)
        t = 0 # To enumerate the entries of the sparse matrix.
        for l in 1:n_k
            s = Kk[l,:]
            for j in 1:(k+1)
                r = vcat(s[1:(j-1)],s[(j+1):end])
                m = Ip[r]
                t+=1
                rows[t] = l
                cols[t] = m
                vals[t] = 1-2*((j-1)%2)
                #@show s,l,r,m,t,vals[t]
            end
        end
        alpha_k = sparse(rows,cols,vals,n_k,n_p)
        push!(alpha,alpha_k)
    end
    return SimplicialComplex(K,I,alpha)
end

In [ ]:
"""
Takes as input a graph g and returns the list of I_k for k=0,1,2.
"""
function g_to_3_cliques(g)
    # Generate I_0
    I0 = Dict([i] => i for i in 1:nv(g))    
    
    # Generate I_1
    I1 = Dict()    
    for (l,e) in enumerate(edges(g))
        i = src(e)
        j = dst(e)
        I1[[i,j]] = l
    end
    
    # Generate I_2
    I2 = Dict()
    t = 0
    for i in 1:nv(g)
        for j in neighbors(g,i)
            if i<j
                for k in neighbors(g,j)
                    if j<k && has_edge(g,k,i)
                        t += 1
                        I2[[i,j,k]] = t
                    end
                end
            end
        end
    end
    
    I = OffsetArray([I0,I1,I2],0:2)
    return I
end

In [ ]:
function w_to_f(sc,w)
    #m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    f = zeros(m_1)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = w[j,i]-w[i,j]
    end
    return f
end

In [ ]:
function phi_to_f(sc,w)
    m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    phi = zeros(m_0)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        phi[i] += w[i,j]
        phi[j] += w[j,i]        
    end
    f = zeros(m_1)
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = phi[j]-phi[i]
    end
    return f
end

### Tests

#### Testing `g_to_3_cliques` and `I_to_SC` 

In [ ]:
g = erdos_renyi(6, 0.4)
g = main_component(g)

In [ ]:
I_3c = g_to_3_cliques(g) # We use I123 instead of I since in Julia I is a reserved name for the identity matrix.

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
sc.K[1]

In [ ]:
sc.I[1]

In [ ]:
sc.K[2]

In [ ]:
sc.I[2]

In [ ]:
sc.alpha[1]

In [ ]:
sc.alpha[2]

In [ ]:
nnz_alpha(sc)

#### Testing `w_to_f` and `phi_to_f`

In [ ]:
(g,w) = redes_wg[("Italy", "crossing")]

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
f = w_to_f(sc,w)

In [ ]:
f = phi_to_f(sc,w)

# Hodge decomposition of $f$

Tools to compute the Hodge decomposition of a 1-chain $f$.

Recall that any $f\in C_1^*$ admits an unique decomposition $f=s+h+g$, where there exists $w\in C_0^*$ such that 

$$
L_0(w)=L^{\uparrow}_0(w)=d_0^*(d_0(w))=d_0^*(g)=d_0^*(f)
$$

and there exists $u\in C_2^*$ such that

$$
L_2^{\downarrow}(u)=d_1(d_1^*(u))=d_1(s)=d_1(f)
$$

In this way, we can find $w$ by solving $L^{\uparrow}_0(w)=d_0^*(f)$ and $u$ by solving $L_2^{\downarrow}(u)=d_1(f)$. Then, we can find $g=d_0(w)$, $s=d_1^*(u)$ and $h=f-g-s$.

## Matricial form

The component-wise form of equation $d_0^*(d_0(w))=d_0^*(f)$ is

$$
(\alpha_1^T)_r{}^s(\alpha_1)_s{}^qw_q = (\alpha_1^T)_r{}^sf_s
$$

or, matricially

$$
\alpha_1^T \alpha_1 w = \alpha_1^T f \;\;\;\; (1)
$$

Similarly, the component-wise form of equation $g = d_0(w)$ is

$$
(\alpha_1)_r{}^s w_s
$$

or, matricially

$$
g = \alpha_1 w
$$

Similarly, the matricial form of equation $d_1(d_1^*(u))=d_1(f)$ is

$$
\alpha_2 \alpha_2^T u = \alpha_2 f \;\;\;\; (2)
$$

and, the matricial form of equation $s=d_1^*(u)$ is

$$
s = \alpha_2^T u
$$

### Test example of Eqs. (18) and (20) in [2].

Eq. (18) of [2] is the analogous of Eq. (1) where $w$ in Eq. (1) is $\alpha$ in Eq. (18) of [2] and $f$ in Eq. (1) is the analogous of $w$ in Eq. (18) of [2].
The matrix of Eq. (18) in [2] is analogous to our $L_0=\alpha_1^T\alpha_1$.
Similarly, Eq. (20) of [2] is the analogous of Eq. (2) where $u$ in Eq. (2) is the analogous of $\gamma$ in Eq. (20) of [2] and $f$ in Eq. (2) is the analogous of $w$ in Eq. (20) of [2].
The matrix of Eq. (20) in [2] is analogous to our $L_2^{\downarrow}=\alpha_2\alpha_2^T$.

The $k$-clique complex of $g$ is the simplicial complex obtained by considering all the $q$-cliques of $g$ for $q\leq k$.

First, the 3-clique complex of the network of Fig. (1) in [2] is created.

In [ ]:
I0 = Dict{Any,Int}()
I1 = Dict{Any,Int}()
I2 = Dict{Any,Int}()

for i in 1:8
    enum!(I0,[i])
end
I0

enum!(I1,[1,2])
enum!(I1,[1,8])
enum!(I1,[2,3])
enum!(I1,[2,6])
enum!(I1,[3,4])
enum!(I1,[3,5])
enum!(I1,[3,6])
enum!(I1,[3,8])
enum!(I1,[4,5])
enum!(I1,[4,6])
enum!(I1,[5,6])
enum!(I1,[6,7])
enum!(I1,[7,8])
I1

enum!(I2,[2,3,6])
enum!(I2,[3,4,5])
enum!(I2,[3,4,6])
enum!(I2,[3,5,6])
enum!(I2,[4,5,6])
I2

In [ ]:
I_vs_k = 

sc = I_to_SC(I_vs_k)
sc.alpha[2]

In [ ]:
#w = zeros(dim_Ck(sc,0))
f = zeros(m_k(sc,1))

I1 = sc.I[1]

f[I1[[1,2]]] = -1   # 1
f[I1[[1,8]]] = 4.2  # 2
f[I1[[2,3]]] = -2   # 3
f[I1[[2,6]]] = 8.1  # 4
f[I1[[3,4]]] = 3.1  # 5
f[I1[[3,5]]] = 5.9  # 6
f[I1[[3,6]]] = 9.8  # 7
f[I1[[3,8]]] = 7.1  # 8
f[I1[[4,5]]] = 3.1  # 9
f[I1[[4,6]]] = 6.9  # 10
f[I1[[5,6]]] = 4.1  # 11
f[I1[[6,7]]] = -1   # 12
f[I1[[7,8]]] = -2   # 13

f

In [ ]:
alpha_1 = sc.alpha[1]

In [ ]:
alpha_1'

Here, $z:=d_0^*(f)$ should equal the r.h.s. of Eq. (18) in [2].

In [ ]:
z = alpha_1'*f

Here, $L_0$ should equal the matrix at the l.h.s. of Eq. (18) in [2].

In [ ]:
L0 = alpha_1'*alpha_1

In [ ]:
w = L0 \ z

Next vector should be equal to the vector $\alpha$ reported in Table 5 of [2].

In [ ]:
w .- w[1]

Next vector should equal the r.h.s. of Eq. (18) in [2].

In [ ]:
L0*w

Check that vector $g=d_0(w)=\alpha_1 w$ equals the vector $w_g$ of Table 4 in [2].

In [ ]:
g = alpha_1*w

Check equality up to numerical tolerance that $d_1(f)=d_1(g)$.

In [ ]:
norm( alpha_1'*f - alpha_1'*g )

In [ ]:
alpha_2 = sc.alpha[2]

Check the form of $L_2^{\downarrow}=\alpha_2\alpha_2^T$. It should equal the matrix at the l.h.s. of Eq. (20) in [2].

In [ ]:
L2d = alpha_2 * alpha_2'

Here, $y:=d_1(f)$ should equal the r.h.s. of Eq. (20) in [2].

In [ ]:
y = alpha_2 * f

In [ ]:
# See 
#    https://stackoverflow.com/questions/38486697/solve-a-particular-linear-system-efficiently-in-julia
#    https://iterativesolvers.julialinearalgebra.org/dev/
#    https://jso.dev/Krylov.jl/dev/solvers/ls/
# on different alternatives about how to solve the linear systems in consideration.

#u = L2d \ y
#u = pinv(Symmetric(Matrix(L2d)))*y
#u = IterativeSolvers.cg(L2d,y)     # this is faster than lsmr in the example, according to @btime
#u = IterativeSolvers.minres(L2d,y)
#u = IterativeSolvers.lsqr(L2d,y)
#u = IterativeSolvers.lsmr(L2d,y)
#u = Krylov.cg(L2d,y)[1]    
#u = Krylov.lslq(L2d,y)[1]
#u = Krylov.lsqr(L2d,y)[1]
u = Krylov.minres_qlp(L2d,y)[1]

The vector $u$ obtained is different from the vector $\gamma$ reported in Table (5) of [2], which is the following one:

In [ ]:
u_johnson = [-0.08 -1.1309 1.2359 -1.1759 1.2809]'

Is the vector $u_{\mathrm{johnson}}$ (i.e. $\gamma$) reported in [2] also a solution of Eq. (20) of [2]? We check:

In [ ]:
norm(L2d*u_johnson - y)

It actually is! What about our solution? Let us check:

In [ ]:
norm(L2d*u - y)

It is as well! Hence, there are multiple solutions, so $L_2^{\downarrow}(u)=y$ is a degenerate (i.e. underdetermined) linear system. 

Only one of the solutions of degenerate linear systems of a positive semidefinite linear operator has minimum norm $|x|$. Therefore, we ask ourselves which of the solutions to $L_2^{\downarrow}u=y$ we have found, has the smallest norm? Let us see:

In [ ]:
norm(u)

In [ ]:
norm(u_johnson)

So our solution has smaller norm than that of [2]. Hence, it is a better candidate for the unique solution of minimum norm.

Next, compute $s=d_1^*(u)=\alpha_2^T u$.

In [ ]:
s = alpha_2'*u

Check the "equality" of $\alpha_2 s = d_1(s) = d_1(f) = \alpha_2 f$.

In [ ]:
norm( alpha_2*s - alpha_2*f )

Compute $h$.

In [ ]:
h = f-s-g

In [ ]:
L1u = alpha_2'*alpha_2
L1d = alpha_1*alpha_1'
L1 = L1u + L1d

Finally check that $L_1(h)=0$.

In [ ]:
norm(L1*h)

## Compute Hodge decomposition

The linear equations that should be solved are numerically challenging. Hence, we use the **MINRES-QLP** algoritm, which is specifically devised to solve least square problems of singular or nearly singular linear equations  [9,10,11,12].

In [ ]:
struct HodgeDecomposition
    f::Array{Real}
    s::Array{Real}
    h::Array{Real}
    g::Array{Real}
    u::Array{Real}
    w::Array{Real}    
    k::Int    
end

In [ ]:
function compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    if k<kappa(sc)
        alpha = sc.alpha[k+1]
        y = alpha*f
        L = alpha*alpha'
        u = Krylov.minres_qlp(L,y)[1]
        #u = Krylov.lsqr(L,y)[1]
        s = alpha'*u
        return s,u
    else
        s = zeros(m_k(sc,k)) # s = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        u = zeros(m_k(sc,k+1))        
        return s,u
    end
end

In [ ]:
function compute_k_th_gradient_hodge_decomposition(sc,f,k)
    if k>=1
        alpha = sc.alpha[k]
        z = alpha'*f
        L = alpha'*alpha
        w = Krylov.minres_qlp(L,z)[1]
        #w = Krylov.lsqr(L,z)[1]
        g = alpha*w
        return g,w
    else
        g = zeros(m_k(sc,k)) # g = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        w = zeros(m_k(sc,k-1))        
        return g,w
    end
end

In [ ]:
function compute_k_th_hodge_decomposition(sc,f,k)
    @assert length(f) == m_k(sc,k)
    s,u = compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    g,w = compute_k_th_gradient_hodge_decomposition(sc,f,k)
    h=f-s-g
    return HodgeDecomposition(f,s,h,g,u,w,k)
end

### Test

In [ ]:
g = erdos_renyi(10, 0.4)

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
for k in 0:kappa(sc)
   println("m_$k=$(m_k(sc,k))")
end

Next, we create a 1-chain $f$ of components $f_{ij}=w_i-w_j$ where $w$ is the 0-chain of components $w_i=i$.

In [ ]:
m0=m_k(sc,0)
m1=m_k(sc,1)
w = collect(1:m0)
f = zeros(m1)
K1 = sc.K[1]
for l in 1:m1
    i,j = K1[l,:]
    f[l] = w[i]-w[j]
end

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,w,0)

In [ ]:
hd.g

In [ ]:
hd.w

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w .- minimum(hd.w)

In [ ]:
u = collect(1:m_k(sc,2))

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,u,2)

In [ ]:
hd.s

# Generating true ratings and rankings

Tools to generate **true** ratings and rankings for a given network.

In [ ]:
"""
Generator of artificial ratings.
    agents : iterable over the agents.
    noise : Gaussian additive noise
    scale : sepparation scale between ratings
"""
function gen_artificial_ratings(agents,noise=0.0,scale=1.0)
    #@show agents
    N = length(agents)
    w = collect(agents)+noise*randn(N)
    w = ratings(N .- w)
    return scale*w
end

In [ ]:
"""
Takes a simplicial complex and a vector of ratings and generates the 1-cochain

    f_{ij} = w_i - w_j + noise*randn()
"""
function gen_artificial_f(sc,w,noise=0.0)
    m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    @assert length(w) == m_0
    f = zeros(m_1)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = w[j]-w[i] + noise*randn()
    end
    return f
end

In [ ]:
"""
Takes a vector and shifts the values of its components by a constant to make all of them non-negative.
"""
function ratings(w)
    return w .- minimum(w)
end

In [ ]:
"""
Normalizes a vector of ratings to have values between 0 and 1 inclusive.
"""
function normalized_ratings(w)
    minw = minimum(w)
    #maxw = maximum(w)
    #return (w .- minw)/(maxw-minw)
    return (w .- minw)/length(w)
end

In [ ]:
"""
Takes a vector of ratings and computes the corresponding ranking
"""
function ranking(w)
    return sortperm(w,rev=true)
end

In [ ]:
"""
Given two vectors v and u, this computes the absolute differences

    x_i = |v_i-u_i|

And returns the vector w of cumulative diferences

    w_i = sum_{j <= i} x_j

This can be used to compare two vectors of ratings or rankings
"""
function compute_cumulative_abs_diff(v,u)
    n = length(v)
    @assert length(u) == n
    cumulative_abs_diff = zeros(n)
    sum = 0
    for i in 1:n
         sum += abs.(v[i]-u[i])
         cumulative_abs_diff[i] = sum
    end
    return cumulative_abs_diff
end

### Test

In [ ]:
g = erdos_renyi(6, 0.4)

In [ ]:
w = gen_artificial_ratings(1:nv(g))

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
ratings(w .- 5)

In [ ]:
ranking(w)

In [ ]:
normalized_ratings(w)

# Inference of ratings and rankings

Tools to compute the inference of ratings and rankings for ensemble of networks.

The computation of Hodge's decomposition can be numerically challenging, specially for large simplicial complexes.
Here we devise several tests to evaluate the performance of different algorithms in solving for $s$, $h$ and $g$ (or equivalently, solving for $u$ and $w$).

We evaluate the performance on different tests:

1. A 1D-lattice of $N$ nodes and $2z$ neithbors per node.
2. Erdos-Renyi random networks of $N$ nodes and connectivity $p=\langle k\rangle/(N-1)$.
3. Barabasi-Albert scale-free networks of $N$ nodes and $q$ new links per new node in the growing algorithm.
4. Watts-Strogatz small-world networks of $N$ nodes, $k$ links per node and a fraction $p$ of randomized links.

### IMPORTANT

For each network model, there is one cell (perhaps more than one) to perform the heavy computations where the inference of ratings and rankings is obtained for 5000 network samples.
These cells are "commented" (actually, the type of the cells are changed to "raw") to prevent running them accidentally, because it is slow.
These cells should be run only once to generate the samples, which are saved into a corresponding **JLD2** file that can be quickly read afterwards for the analysis of the generated data.

### Color schemes

In [ ]:
palette_colors = palette(:auto)
col1 = palette_colors[1]
col2 = palette_colors[2]
col3 = palette_colors[3]
col4 = palette_colors[4]
col5 = palette_colors[5]
col6 = palette_colors[6]
col7 = palette_colors[7]
col8 = palette_colors[8]
cols = [col1,col2,col3,col4,col5,col6,col7,col8]

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,6)
cols1 = [col1,col2]
colors_N = palette(cols1,6)

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,8)
#colors_z = palette(:hawaii,13)
cols2 = [col4,col6]
colors_z = palette(cols2,13)
colors_k = colors_z
colors_q = colors_z
colors_p = colors_z

### 1D-lattice

#### Disorder parameter vs noise

In [ ]:
#### This cell loads the .jld2 file with the JLD2 dictionary with the samples. 
#### If the file to be laoded doesn't exists, then it creates and empty dictionary to be filled later by running the simulation.

filename = "data/lattice1D_stats_vs_sigma.jld2"

# Deserialize data
if isfile(filename)
    #@load filename grid1D_stats_vs_sigma
    #lattice1D_stats_vs_sigma = grid1D_stats_vs_sigma
    @load filename lattice1D_stats_vs_sigma
else    
    lattice1D_stats_vs_sigma = Dict()    
end

### Perform the analysis for the 1D lattices

Here, we create the figures to analyze the simulations computed for the 1D lattices.

##### $\tau$ varying $N$

In [ ]:
fig2a_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}\langle\bar{\tau}\rangle")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 1
       continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    
    scatter!(
        list_sigma,
        list_N_tau_mean/N^1.5,#/N^1.35/log(N),
        #yerr=list_N_tau_std/N,
        label="\$N=$N\$",
        markersize=2.0,
        markerstrokewidth=0.0,
        c=colors_N[c]
    )
end  
plot!(x->0.53*x+0.1,c=:black,style=:dash,width=3.0,label=L"0.53\,\sigma+0.1")
#plot!(x->0.53*x+0.1,c=:gray,style=:dash,width=3.0,label="")
#savefig("figs/fig2a.pdf")
plot!()

##### $\tau$ varying $z$

In [ ]:
fig2a_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"z^{1.3}N^{-1/2}\langle\bar{\tau}\rangle")
#plot!(0.0:0.0,x->0.0,label="\$N=512\$",width=0.0,c=:white)
plot!(legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
)

colors_z = palette(cols2,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
       continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    
    scatter!(
        list_sigma,
        #list_N_tau_mean/N^1.5,
        #list_N_tau_mean/N^1.5,#/N^1.35/log(N),
        list_N_tau_mean/N^1.5*z^1.3,
        #yerr=list_N_tau_std/N,
        label="\$z=$z\$",
        markersize=2.0,
        markerstrokewidth=0.0,
        c=colors_z[c]
    )
end  
plot!(x->0.6*x+0.1,c=:black,style=:dash,width=3.0,label=L"0.6\,\sigma+0.1")
#savefig("figs/fig2b.pdf")
plot!()

##### $\rho$ varying $N$

In [ ]:
fig3a_1 = plot()
#plot!(yscale=:log10)
#plot!(xlim=(0,0.4))
#plot!(ylim=(0,0.02))
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(legend=:bottomright)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,5,10,15],[L"0",L"5",L"10",L"15"]),
    tickfont=font(11),
)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 1
       continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0,
        #yerr=list_N_rho_std/N,
        label="\$N=$N\$",
        #xlims=(0,2),
        #ylims=(1.0e-5,:auto),
        markersize=2.0,
        markerstrokewidth=0.0,    
        c=colors_N[c]
    )
end    

B = bbox(0.26,0.05,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset = (1, B),subplot = 2)

#plot()
plot!(0.0:0.1:0.6,x->0.0,style=:dash,c=:lightgray,width=2.0,subplot = 2)
#plot!(yscale=:log10)
plot!(xlim=(0.01,0.4),subplot = 2)
#plot!(ylim=(-0.01,0.02),subplot = 2)
plot!(ylim=(-1.0,2.0),subplot = 2)
#plot!(ylim=(1.0e-5,1.0e3))
#plot!(xticks=[0.1,0.3],subplot=2)
plot!(xlabel=L"\sigma",subplot = 2)
plot!(ylabel=L"10^{2}\langle\bar{\rho}\rangle",subplot = 2)
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    subplot=2,
    xticks=([0.1,0.3],[L"0.1",L"0.3"]),
    yticks=([-1,0,1,2],[L"-1",L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 1
        continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0*100,
        #yerr=list_N_rho_std/N,
        label="\$N=$N\$",
        #xlims=(0,2),
        #ylims=(1.0e-5,:auto),
        markersize=2.0,
        markerstrokewidth=0.0,
        subplot = 2,
        c=colors_N[c]
    )
end    
#plot!(legend=:bottomright)
plot!(legend=false,subplot = 2)

##### $\rho$ varying $z$

In [ ]:
fig3a_2 = plot()
#plot!(yscale=:log10)
#plot!(xlim=(0,0.4))
#plot!(ylim=(0,0.02))
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
#plot!(0.0:0.0,x->0.0,label="\$N=512\$",width=0.0,c=:white)
plot!(legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,5,10,15],[L"0",L"5",L"10",L"15"]),
    tickfont=font(11),
)

colors_z = palette(cols2,6)
colors_z_aux = copy(colors_z[:])
colors_z_aux[6] = RGB{Float64}(1.0,0.0,0.0)

c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
       continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    scatter!(

        list_sigma,
        list_N_rho_mean/N^1.0*z^1.0,
        #yerr=list_N_rho_std/N,
        label="\$z=$z\$",
        #xlims=(0,2),
        #ylims=(1.0e-5,:auto),
        markersize=2.0,
        markerstrokewidth=0.0,    
        #c=colors_k[c],
        c=colors_z_aux[c],
    )
end    
plot!(legend=:bottomright)

B = bbox(0.2,0.05,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset = (1, B),subplot = 2)
#plot()
#plot!(yscale=:log10)
plot!(xlim=(0.01,1.1),subplot = 2)
#plot!(ylim=(-0.001,0.02),subplot = 2)
plot!(ylim=(-0.1,2.0),subplot = 2)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma",subplot = 2)
plot!(ylabel=L"10^2\langle\bar{\rho}\rangle",subplot = 2)
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
        continue 
    end
    c+=1
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0*100,
        #yerr=list_N_rho_std/N,
        label="\$N=$N\$ \$z=$z\$",
        #xlims=(0,2),
        #ylims=(1.0e-5,:auto),
        markersize=2.0,
        markerstrokewidth=0.0,
        subplot = 2,
        #c=colors_z[c],
        c=colors_z_aux[c],
    )
end    
#plot!(0.0:0.1:0.6,x->0.0,style=:dash,c=:black,width=2.0,subplot = 2)
#plot!(legend=:bottomright)
#plot!(xticks = 0:0.5:1.2,subplot=2)
#plot!(yticks = 0:0.01:0.02,subplot=2)
#plot!(yticks = 0:1.0:2.0,subplot=2)
plot!(legend=false,subplot = 2)
plot!(
    subplot=2,
    xticks=([0.5,1.0],[L"0.5",L"1.0"]),
    yticks=([0,1,2],[L"0",L"1",L"2"]),
    tickfont=font(11),
)

plot!()

##### $\sigma_c$

In [ ]:
function find_first_nonzero(x)
    for i in 1:length(x) 
        if x[i]>0.0
            return i
        end
    end
    return length(x)
end

In [ ]:
lattice1D_peaks = Dict()

for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    #if k != 1.0
    #    continue
    #end
    #@show ky
    
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
   
    plt = plot()
    plot!(legendtitle=L"\mathrm{1D}",legendtitlefontsize=8)
    #plot!(xlim=(0.0,2.0))
    #plot!(ylim=(-0.5,0.5))
    #plot!(ylim=(-0.5,:auto))
    plot!(xlabel=L"\sigma")
    plot!(ylabel=L"\langle\bar{\rho}\rangle")    

    r_star = find_first_nonzero(list_N_rho_mean)-1
    sigma_star = list_sigma[r_star]
    #@show t_star,sigma_star
    
    list_r = 2*r_star:length(list_sigma)
    list_sigma_c_vs_r        = []
    list_sigma_c_err_vs_r    = []
    list_d2rho_peak_vs_r     = []
    list_d2rho_peak_err_vs_r = []
    list_mresid_vs_r         = []
    bestfit_r                = 0
    bestfit_list_sigma       = 0
    bestfit_p                = 0
    bestfit_sigma_c          = 0
    bestfit_sigma_c_err      = 0
    bestfit_d2rho_peak       = 0
    bestfit_d2rho_peak_err   = 0
    #p = [10.0,1.0,0.25,0.0] # k x0 A C
    p = [1.0,0.25,10.0,0.0] # p1=x0 p2=A p3=B p4=C 
    for r in list_r
        p0 = p
        fit_range = 1:r
        fit_list_sigma = collect(list_sigma[fit_range])
        fit_list_N_rho_mean = list_N_rho_mean[fit_range]
        fit = LsqFit.curve_fit(psigmoid,fit_list_sigma,fit_list_N_rho_mean/N,p0)   
        mresid = norm(fit.resid)/r # mean residuals
        p = fit.param
        #@show N,z,q,mresid,p
        p_err = LsqFit.standard_errors(fit)
        #for i in 1:length(p)
        #    pi = p[i]
        #    pe = p_err[i]
        #end
        A=p[2]
        B=p[3]
        A_err=p_err[2]
        B_err=p_err[3]
        sigma_c          = p[1]
        sigma_c_err      = p_err[1]
        d2rho_peak       = A*B/4 #dsigmoid(sigma_c,fit.param)
        d2rho_peak_err   = sqrt((B/4*A_err)^2+(A/4*B_err)^2) #estimate_error(dsigmoid,sigma_c,fit)
        push!(list_sigma_c_vs_r,sigma_c)
        push!(list_sigma_c_err_vs_r,sigma_c_err)
        push!(list_d2rho_peak_vs_r,d2rho_peak)
        push!(list_d2rho_peak_err_vs_r,d2rho_peak_err)
        push!(list_mresid_vs_r,mresid)
        if d2rho_peak >= bestfit_d2rho_peak
            bestfit_r              = r
            bestfit_list_sigma     = fit_list_sigma
            bestfit_p              = p
            bestfit_sigma_c        = sigma_c
            bestfit_sigma_c_err    = sigma_c_err
            bestfit_d2rho_peak     = d2rho_peak
            bestfit_d2rho_peak_err = d2rho_peak_err
        end
    end      
    @show N,z,bestfit_r,bestfit_sigma_c,bestfit_d2rho_peak
    
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0,
        label="\$N=$N\$, \$z=$z\$",
        markersize=2.0,
        markerstrokewidth=0.0,        
    )
    plot!(bestfit_list_sigma,x->psigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle",width=2.0,color=:black,style=:dash)    
    plot!(bestfit_list_sigma,x->sigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle'",width=2.0)
    plot!(bestfit_list_sigma,x->dsigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle''",width=2.0)
    #scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\bar{\rho}\rangle''(\sigma_c)",markersize=4.0,markerstrokewidth=0.0)
    scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\tilde{\rho}\rangle",markersize=4.0,markerstrokewidth=0.0)    
    scatter!([sigma_star],[0.0],label=L"\sigma^*",markersize=4.0,markerstrokewidth=0.0,c=:red,cl=:red,mc=:red)
    plot!(legend=:topright)
    
    B = bbox(0.1,0.01,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    plot!(plt,inset = (1, B),subplot = 2)
    #plot!(xlabel=L"r",subplot=2)
    plot!(xlabel=L"\sigma^{**}",subplot=2)
    plot!(ylabel=L"\sigma_c",subplot=2,yguidefont = font(12,color=col6),)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],label="",subplot=2,c=col6,lc=col6,mc=col6)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    plot!(list_sigma[list_r[1:10:end]],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    #plot!(list_r,list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)    
    plot!(list_sigma[list_r],list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    #@show bestfit_r
    #scatter!([bestfit_r],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    scatter!([list_sigma[bestfit_r]],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    
    plt2 = twinx(plt[2])
    plot!(
        plt2,
        #xlabel=L"N",
        #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
        ylabel=L"\langle\tilde{\rho}\rangle",
        yguidefont = font(12,color=col7),
        #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
        #xscale=:log10,
        #yscale=:log10,
        subplot=2,
    )
    #scatter!(plt2,list_r[1:10:end],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    scatter!(plt2,list_sigma[list_r[1:10:end]],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    #plot!(plt2,list_r,list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    plot!(plt2,list_sigma[list_r],list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    
    lattice1D_peaks[ky] = [plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star]    
    savefig("figs/SM-lattice1D-sigma_c-N$N-z$z.pdf")

end  
lattice1D_peaks[(2048,1)][1]

###### $\sigma_c$ vs $N$

In [ ]:
fig4a_1 = plot()
#plot!(xlim=(0.0,5.5))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
plot!(xscale=:log10)
plot!(legend=:bottomright,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)
plot!(
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"10^6",L"10^7",L"10^8",L"10^9",L"10^{10}",L"10^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([1,1.05,1.1],[L"1.00",L"1.05",L"1.10"]),
    tickfont=font(11),
)

list_N                   = []
list_sigma_c_vs_N        = []
list_sigma_c_err_vs_N    = []
list_d2rho_peak_vs_N     = []
list_d2rho_peak_err_vs_N = []
for ky in sort(collect(keys(lattice1D_peaks)))
    (N,z) = ky    
    if z != 6
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = lattice1D_peaks[ky]
    push!(list_N,N)
    push!(list_sigma_c_vs_N,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_N,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_N,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_N,bestfit_d2rho_peak_err)
    @show bestfit_d2rho_peak,bestfit_d2rho_peak_err
end
@show list_N,list_sigma_c_vs_N,list_d2rho_peak_vs_N

plot!(ylim=(1.0,1.11))
#plot!(yticks = 1.0:0.05:1.1)
plot!(xlabel=L"N")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_N,list_sigma_c_vs_N,yerr=list_sigma_c_err_vs_N,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_N,list_sigma_c_vs_N,label="",c=col6,lc=col6,mc=col6)

fig4a_1_t = twinx(fig4a_1)
plot!(fig4a_1_t,xscale=:log10)
plot!(
    fig4a_1_t,
    #xlabel=L"N",
    #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    #xscale=:log10,
    #yscale=:log10,
)
plot!(fig4a_1_t,ylim=(0.4,0.51))
plot!(fig4a_1_t,yticks = 0.4:0.05:0.5)
plot!(fig4a_1_t,list_N,list_d2rho_peak_vs_N,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4a_1_t,list_N,list_d2rho_peak_vs_N,yerr=list_d2rho_peak_err_vs_N,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    fig4a_1_t,
    yticks=([0.4,0.45,0.5],[L"0.40",L"0.45",L"0.50"]),
    tickfont=font(11),
)

plot!()

###### $\sigma_c$ vs $z$

In [ ]:
fig4a_2 = plot()
plot!(legend=:bottom,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)

list_z                   = []
list_sigma_c_vs_z        = []
list_sigma_c_err_vs_z    = []
list_d2rho_peak_vs_z     = []
list_d2rho_peak_err_vs_z = []
for ky in sort(collect(keys(lattice1D_peaks)))
    (N,z) = ky    
    if N != 512
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = lattice1D_peaks[ky]
    push!(list_z,z)
    push!(list_sigma_c_vs_z,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_z,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_z,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_z,bestfit_d2rho_peak_err)
    @show bestfit_d2rho_peak,bestfit_d2rho_peak_err
end
#@show list_z,list_sigma_c_vs_z,list_d2rho_peak_vs_z

cola=:cyan
colb = :magenta

@. fm(x,p) = p[1]*x^p[2]
fit_fm = LsqFit.curve_fit(fm,list_z[2:end],list_sigma_c_vs_z[2:end],[0.5,0.5])
@show fit_fm.param
@show standard_errors(fit_fm)

#plot!(xticks=([1,2,4],[L"2^0",L"2^1",L"2^2"]))
#plot!(yticks=([2e-2,2e-1,2e0],[L"2^{-2}",L"2^{-1}",L"2^0"]))
plot!(
    xticks=([1,2,4],[L"2^0",L"2^1",L"2^2"]),
    yticks=([2e-2,2e-1,2e0],[L"2^{-2}",L"2^{-1}",L"2^0"]),
    tickfont=font(11),
)
plot!(ylims=(2e-1,2))
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(xlabel=L"z")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_z[2:end],x->fm(x,fit_fm.param),label=L"\sim z^{0.466\pm 0.003}",c=cola,lc=cola,mc=cola,style=:dash,width=3.0)
plot!(list_z,list_sigma_c_vs_z,yerr=list_sigma_c_err_vs_z,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_z,list_sigma_c_vs_z,label="",c=col6,lc=col6,mc=col6)

plot!(1.0:0.1:1.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim z^{-0.98\pm 0.01}")

fit_fm = LsqFit.curve_fit(fm,list_z[2:end],list_d2rho_peak_vs_z[2:end],[2.0,-1.0])
@show fit_fm.param
@show standard_errors(fit_fm)

@show list_d2rho_peak_vs_z[2:end]

fig4a_2_t = twinx(fig4a_2)
plot!(
    fig4a_2_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    yscale=:log10,
    yticks=([2e-2,2e-1,2e0,2e1],[L"2^{-2}",L"2^{-1}",L"2^{-0}",L"2^{1}"]),
    tickfont=font(11),
    ylims=(2e-2,2e1),
)
plot!(fig4a_2_t,list_z[2:end],x->fm(x,fit_fm.param),label="",c=colb,lc=colb,mc=colb,style=:dash,width=3.0)
plot!(fig4a_2_t,list_z,list_d2rho_peak_vs_z,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4a_2_t,list_z,list_d2rho_peak_vs_z,yerr=list_d2rho_peak_err_vs_z,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)

plot!()

##### $|f|$ varying $N$

In [ ]:
figSM9a_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)
#plot!(yscale=:log10)
plot!(ylim=(8.6,16.2))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|f|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([1,2,3,4,5],[L"1",L"2",L"3",L"4",L"5"]),
    yticks=([10,12,14,16],[L"10",L"12",L"14",L"16"]),
    tickfont=font(11),
)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 6
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        #list_norm_f_mean,
        list_norm_f_mean/N^0.5,
        #list_norm_f_mean/N^1.5,
        #yerr=list_norm_f_std,
        label="\$N=$N\$",
        #xlims=(0,2),
        #ylims=(1.0e-5,:auto),
        markersize=3.0,
        markerstrokewidth=0.0,
        c=colors_N[c]
    )
end    
plot!()

##### $|f|$ varying $z$

In [ ]:
figSM9a_2 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"z^{-1}\sigma")
plot!(ylabel=L"z^{-3/2}N^{-1/2}|f|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
list_z = []
list_norm_f_mean_at_sigma0 = []
#alpha=0.87
#beta=1.37
#alpha=0.9
#beta=1.4
alpha=1.0
beta=1.5
@show a = alpha/beta
@show b = 1.0/beta
colors_z = palette(cols2,6)
colors_z_aux = Vector{Any}(undef,length(colors_z))
colors_z_aux .= colors_z
colors_z_aux[end] = :red
cols_aux = Vector{Any}(undef,length(colors_z))
cols_aux .= cols[1:length(cols_aux)]
cols_aux[end] = :black
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512 #|| z < 4
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    push!(list_z,z)
    push!(list_norm_f_mean_at_sigma0,list_norm_f_mean[1]/N^0.5)
    #plot!(
    scatter!(        
        list_sigma*z^(-alpha),
        list_norm_f_mean/N^0.5*z^(-beta),
        #yerr=list_norm_f_std,
        label="\$z=$z\$",
        markersize=2.0,
        markerstrokewidth=0.0,
        #c=colors_z[c],
        #c=colors_z_aux[c],
        c=cols_aux[c],
        xlim=(-0.1,5.0),
        ylim=(-0.1,5.0),
    )
end    
plot!()

In [ ]:
@show list_z
@show list_norm_f_mean_at_sigma0

#p = [0.72,1.5,1e-3,1.0]
#@. fm(x,p) = p[1]-p[2]*exp(-p[3]*x)*x^(-p[4])

#fit_p = [0.72,1e-3,1.0]
#@. fm(x,p) = p[1]*(1-exp(-p[2]*x)*x^(-p[3]))

#p = [0.72,1.5,1.0]
#@. fm(x,p) = p[1]-p[2]*x^(-p[3])

#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^(-p[2])

p = [1.0,1.0,1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]*(1+p[3]*x^p[4])
fit_fm = LsqFit.curve_fit(fm,list_z,list_norm_f_mean_at_sigma0,p)

@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(list_z,list_norm_f_mean_at_sigma0,label="")
plot!(list_z,x->fm(x,p),label=L"(0.414\pm 0.001)z^{0.503\pm 0.002}(1+(1.412\pm 0.006)z^{0.989\pm 0.002})")
plot!(
  #yscale=:log10,
  #xscale=:log10,
  xlabel=L"z",
  ylabel=L"N^{-1/2}|f|(\sigma=0)",
)

##### $|s|$ varying $N$

In [ ]:
figSM10a_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|s|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,5,10],[L"0",L"5",L"10"]),
    tickfont=font(11),
)
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 6
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_s_mean/N^0.5,
        #yerr=list_norm_s_std,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,
        c=colors_N[c]
    )
end  
plot!(x->2.25*x+0.3,c=:black,style=:dash,width=3.0,label=L"2.25\,\sigma+0.3")
plot!()

##### $|s|$ varying $z$

In [ ]:
figSM10a_2 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"z^{-5/8}N^{-1/2}|s|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
)
colors_z = palette(cols2,6)
colors_z_aux = Vector{Any}(undef,length(colors_z))
colors_z_aux .= colors_z
colors_z_aux[end] = :red
#colors_z = palette(cols2,6)
#cols_aux = Vector{Any}(undef,length(colors_z))
#cols_aux .= cols[1:length(cols_aux)]
#cols_aux[end] = :black
list_z = []
list_norm_s_mean_end = []
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    push!(list_z,z)
    push!(list_norm_s_mean_end,list_norm_s_mean[end]/N^0.5)
    scatter!(
        list_sigma,
        #list_norm_s_mean/N^0.5,
        #list_norm_s_mean/N^0.5/z^0.5,
        list_norm_s_mean/N^0.5/z^(5/8),
        #yerr=list_norm_s_std,
        label="\$z=$z\$",
        markersize=2.0,
        markerstrokewidth=0.0,
        #c=colors_z[c],
        c=colors_z_aux[c],
        #c=cols_aux[c],
    )
end    
plot!(x->0.725*x+0.1,style=:dash,c=:black,label=L"0.725\sigma+0.1",width=3.0)
plot!()

In [ ]:
i_zmin = 4
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]

#i_zmin = 1
#p = [1.0,1.0,1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]*(1+p[3]*x^p[4])

#i_zmin = 1
#p = [1.0,1.0,1.0,1.0,1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*x^p[3]*log(p[4]+x^p[5])

#i_zmin = 1
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]*log(x)

#i_zmin = 1
#p = [1.0,0.0]
#@. fm(x,p) = p[1]*log(p[2]+x)

fit_fm = LsqFit.curve_fit(fm,list_z[i_zmin:end],list_norm_s_mean_end[i_zmin:end],p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

@show list_z
@show list_norm_s_mean_end

plot()
scatter!(
    list_z[1:end],
    list_norm_s_mean_end[1:end],
    #xscale=:log10,
    #yscale=:log10,
    xlim=(0.9,7),
    #xlim=(0.1,7),
    #ylim=(0.01,100),
    label="",
    legend=false,
    xlabel=L"z",
    ylabel=L"N^{-1/2}|s|(\sigma=5)",
)
plot!(
    #list_z[i_zmin:end],
    logrange(list_z[i_zmin],1.01,list_z[end]),
    x->fm(x,p),
)

In [ ]:
5.0/8.0

##### $|h|$ varying $N$

In [ ]:
figSM11a_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^6|h|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,2,4,6],[L"0",L"2",L"4",L"6"]),
    tickfont=font(11),
)
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 6
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_h_mean*1e6,
        #yerr=list_norm_s_std,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,
        c=colors_N[c]
    )
end    
plot!()

##### $|h|$ varying $z$

In [ ]:
figSM11a_2 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^6|h|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4],[L"0",L"1",L"2",L"3",L"4"]),
    tickfont=font(11),
)
colors_z = palette(cols2,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_h_mean*1e6,
        #yerr=list_norm_s_std,
        label="\$z=$z\$",
        markersize=3.0,
        markerstrokewidth=0.0,
        c=colors_z[c]
    )
end    
plot!()

##### $|g|$ varying $N$

In [ ]:
figSM12a_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([9,10,11],[L"9",L"10",L"11"]),
    tickfont=font(11),
)
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
plot!(ylim=(9,11))
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if z != 6
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_g_mean/N^0.5,
        #yerr=list_norm_s_std,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,
        c=colors_N[c]
    )
end    
plot!()

##### $|g|$ varying $z$

In [ ]:
figSM12a_2 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
#plot!(xlabel=L"\sigma")
plot!(xlabel=L"z^{-4/3}\sigma")
plot!(ylabel=L"z^{-4/3}N^{-1/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
#plot!(ylabel=L"N^{-1/2}|g|")
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
#alpha = 1.4
#beta  = 1.4
alpha = 4/3
beta  = 4/3
colors_z = palette(cols2,6)
colors_z_aux = Vector{Any}(undef,length(colors_z))
colors_z_aux .= colors_z
colors_z_aux[end] = :red
cols_aux = Vector{Any}(undef,length(colors_z))
cols_aux .= cols[1:length(cols_aux)]
cols_aux[end] = :black
list_z = []
list_list_norm_g_mean_sigma0 = []
c=0
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z) = ky
    if N != 512
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    push!(list_z,z)
    push!(list_list_norm_g_mean_sigma0,list_norm_g_mean[1]/N^0.5)
    #plot!(
    scatter!(
        list_sigma*z^(-alpha),
        list_norm_g_mean/N^0.5*z^(-beta),
        #yerr=list_norm_s_std,
        label="\$z=$z\$",
        markersize=3.0,
        markerstrokewidth=0.0,
        #c=colors_z[c],
        #c=colors_z_aux[c],
        c=cols_aux[c],
        #xlim=(0.0,1.0),
        #ylim=(0.5,1.0),
    )
end    
plot!()

In [ ]:
@show list_z
@show list_list_norm_g_mean_sigma0

p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]

#p = [1.0,1.0,1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]*(1+p[3]*x^p[4])

i_zmin =2

fit_fm = LsqFit.curve_fit(fm,list_z[i_zmin:end],list_list_norm_g_mean_sigma0[i_zmin:end],p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

@show list_z
@show list_list_norm_g_mean_sigma0

plot()
scatter!(
    list_z,
    list_list_norm_g_mean_sigma0,
    label="",
    xscale=:log10,
    yscale=:log10,
)
plot!(
    list_z[i_zmin:end],
    x->fm(x,p),
    label="",
)

##### $\kappa_k$ vs $N$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13a_1 = plot()
plot!(xlabel=L"N")
plot!(ylabel=L"\langle \kappa_k\rangle")
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xscale=:log10)
plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([1e2,1e3],[L"10^2",L"10^3"]),
    yticks=([1e2,1e3,1e4],[L"10^2",L"10^3",L"10^4"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;z=6",legendtitlefontsize=8)

list_N = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z)=ky
    if z != 6
        continue
    end
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    push!(list_N,N)
    push!(list_m_0_mean,mean(m_0))
    push!(list_m_0_std ,std(m_0))
    push!(list_m_1_mean,mean(m_1))
    push!(list_m_1_std ,std(m_1))
    push!(list_m_2_mean,mean(m_2))
    push!(list_m_2_std ,std(m_2))
end       
@show list_N
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_N,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1,
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_N,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_N,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2,
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_N,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2,
)

scatter!(
    list_N,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3,
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_N,list_m_2_mean,[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm2(x,fit_m2.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3,
)

plot!()

##### $\kappa_k$ vs $z$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13a_2 = plot()
plot!(xlabel=L"z")
plot!(ylabel=L"10^{-3}\langle \kappa_k\rangle")
#plot!(ylim=(1.0e-5,1.0e3))
#plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,6.1))
#plot!(ylim=(1.0,10000.0))
plot!(
    xticks=([1,2,3,4,5,6],[L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([0,2,4,6],[L"0",L"2",L"4",L"6"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{1D},\;N=512",legendtitlefontsize=8)

list_z = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(lattice1D_stats_vs_sigma)))
    (N,z)=ky
    if N != 512
        continue
    end
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = lattice1D_stats_vs_sigma[ky]
    push!(list_z,z)
    push!(list_m_0_mean,mean(m_0))
    push!(list_m_0_std ,std(m_0))
    push!(list_m_1_mean,mean(m_1))
    push!(list_m_1_std ,std(m_1))
    push!(list_m_2_mean,mean(m_2))
    push!(list_m_2_std ,std(m_2))
end       
@show list_z
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_z,
    list_m_0_mean*1e-3,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1,
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_z,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_z,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param)*1e-3,
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_z,
    list_m_1_mean*1e-3,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2,
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_z,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_z,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param)*1e-3,
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto z",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2,
)

scatter!(
    list_z,
    list_m_2_mean*1e-3,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3,
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_z[1:end],list_m_2_mean[1:end],[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    #list_z[1:end],
    list_z[1]:0.1:list_z[end],
    #list_m_0_mean,
    x->fm2(x,fit_m2.param)*1e-3,
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto z^{2.31\pm 0.05}",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3,
)

plot!()

### Erdos-Renyi

#### Developing example

In [ ]:
N = 10000
mean_k = 3.0
p = mean_k/(N-1)
@show N
@show mean_k
@show p
#g = erdos_renyi(6, 0.4)
g = erdos_renyi(N,p)
@show nv(g)
@show ne(g)
@show 2*ne(g)/N
#@show g
;

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="ER")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="ER")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

In [ ]:
sum(abs.(ranking(w)-ranking(hd.w)))

#### Disorder parameter vs noise

In [ ]:
filename = "data/ER_stats_vs_sigma.jld2"

# Deserialize data
if isfile(filename)
    @load filename ER_stats_vs_sigma
else    
    ER_stats_vs_sigma = Dict()    
end

##### $\bar{\tau}$ varying $N$

In [ ]:
fig2b_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\tau}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label="\$\\bar{k}=16\$",width=0.0,c=:white)
plot!(
    xticks=([1,2,3,4,5,6],[L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([0.0,0.5,1.0,1.5],[L"0.0",L"0.5",L"1.0",L"1.5"]),
    tickfont=font(11),
)
plot!(legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)

colors_N = palette(cols1,5)
c = 0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k == 16.0
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        kk = Int(k)
        c+=1
        scatter!(
            list_sigma,
            list_N_tau_mean/N^1.0,
            #yerr=list_N_tau_std/N,
            #label="\$N=$N\$  \$\\bar{k}=$kk\$"
            label="\$N=$N\$",
            markersize=3.0,
            markerstrokewidth=0.0,            
            c=colors_N[c]
        )
    end
end    
plot!(x->0.3*x+0.05,label=L"0.3\,\sigma+0.05",c=:black,style=:dash,width=3)
#savefig("figs/fig2a.pdf")
plot!()

##### $\bar{\tau}$ varying $\bar{k}$

In [ ]:
fig2b_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\bar{k}^{2/3}\,\langle\bar{\tau}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label="\$N=512\$",width=0.0,c=:white)
plot!(
    xticks=([1,2,3,4,5,6],[L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([0,5,10],[L"0",L"5",L"10"]),
    tickfont=font(11),
)
plot!(legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)

colors_k = palette(cols2,13)
colors_k_aux = vcat([c for c in colors_k[1:12]],[:red])
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N==512
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        kk = Int(k)
        c+=1
        scatter!(
            list_sigma,
            list_N_tau_mean/N*k^(2/3),
            #yerr=list_N_tau_std/N,
            #label="\$N=$N\$  \$\\bar{k}=$kk\$",
            label="\$\\bar{k}=$kk\$",
            markersize=3.0,
            markerstrokewidth=0.0,              
            #c=colors_k[c],
            c=colors_k_aux[c],
        )
    end
end    
plot!(x->1.8*x,c=:black,style=:dash,width=3.0,label=L"1.8\,\sigma")
plot!()

##### $\bar{\rho}$ varying $N$

In [ ]:
fig3b_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10\langle\bar{\rho}\rangle")
#plot!(0.0:0.0,x->0.0,label="\$\\bar{k}=16\$",width=0.0,c=:white)
plot!(legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
plot!(
    xticks=([1,2,3,4,5,6],[L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([0,2,4,6,8],[L"0",L"2",L"4",L"6",L"8"]),
    tickfont=font(11),
)
colors_N = palette(cols1,5)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k == 16.0
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        c+=1
        kk=Int(k)
        scatter!(
            list_sigma,
            list_N_rho_mean/N^1.0*10,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$k=$kk\$",
            label="\$N=$N\$",
            markersize=3.0,
            markerstrokewidth=0.0,              
            c=colors_N[c]            
        )
    end
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!(legend=:bottomright)


B = bbox(0.24,0.01,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset = (1, B),subplot=2)

#plot()
#plot!(yscale=:log10)
#plot!(xlim=(-0.05,1.6),subplot=2)
plot!(xlim=(0.5,0.8),subplot=2)
#plot!(ylim=(-0.001,0.002),subplot=2)
plot!(ylim=(-1.0,2.0),subplot=2)
plot!(xlabel=L"\sigma",subplot=2)
plot!(ylabel=L"10^3\langle\bar{\rho}\rangle",subplot=2)
#plot!(ylabel="",subplot=2)
#plot!(0.0:0.0,x->0.0,label="\$\\bar{k}=16\$",width=0.0,c=:white)
plot!(0.5:0.01:0.8,x->0.0,label="",width=2.0,c=:lightgray,subplot=2,style=:dash)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k == 16.0
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        c+=1
        kk=Int(k)
        scatter!(
            list_sigma,
            list_N_rho_mean/N^1.0*1e3,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$k=$kk\$",
            #label="\$N=$N\$",
            label="\$N=$N\$",
            markersize=3.0,
            markerstrokewidth=0.0,              
            c=colors_N[c],
            subplot=2
        )
    end
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!(legend=false,subplot=2)

##### $\bar{\rho}$ varying $\bar{k}$

In [ ]:
fig3b_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(0.0:0.0,x->0.0,label="\$N=512\$",width=0.0,c=:white)
plot!(
    xticks=([1,2,3,4,5,6],[L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
)
plot!(legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)

colors_k = palette(cols2,13)
colors_k_aux = [c for c in colors_k]
colors_k_aux[13] = RGB{Float64}(1.0,0.0,0.0)

c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N == 512
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        c+=1
        kk=Int(k)
        scatter!(
            list_sigma, # .- k^0.66/5.5,
            list_N_rho_mean/N, #*k^0.66,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$k=$kk\$",
            label="\$\\bar{k}=$kk\$",
            markersize=3.0,
            markerstrokewidth=0.0,              
            #c=colors_k[c],
            c=colors_k_aux[c],
            #subplot=2            
        )
    end
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!(legend=:topleft)#,legendtitle="ER, \$N=512\$",legendtitlefontsize=8)

##### $\sigma_c$

In [ ]:
ER_peaks = Dict()

for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    kk=Int(k)    
    #if k != 1.0
    #    continue
    #end
    #@show ky
    
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
   
    plt = plot()
    plot!(legendtitle=L"\mathrm{ER}",legendtitlefontsize=8)
    #plot!(xlim=(0.0,2.0))
    #plot!(ylim=(-0.5,0.5))
    #plot!(ylim=(-0.5,:auto))
    plot!(xlabel=L"\sigma")
    plot!(ylabel=L"\langle\bar{\rho}\rangle")    

    r_star = find_first_nonzero(list_N_rho_mean)-1
    sigma_star = list_sigma[r_star]
    
    list_r = 3*r_star:200
    #list_r = 3*r_star:300
    
    list_sigma_c_vs_r        = []
    list_sigma_c_err_vs_r    = []
    list_d2rho_peak_vs_r     = []
    list_d2rho_peak_err_vs_r = []
    list_mresid_vs_r         = []
    bestfit_r                = 0
    bestfit_list_sigma       = 0
    bestfit_p                = 0
    bestfit_sigma_c          = 0
    bestfit_sigma_c_err      = 0
    bestfit_d2rho_peak       = 0
    bestfit_d2rho_peak_err   = 0
    
    #p = [10.0,1.3,0.25,0.0]
    p = [1.3,0.25,10.0,0.0] # p1=x0 p2=A p3=B p4=C     
    for r in list_r
        p0 = p
        fit_range = r_star:r
        fit_list_sigma = collect(list_sigma[fit_range])
        fit_list_N_rho_mean = list_N_rho_mean[fit_range]
        fit = LsqFit.curve_fit(psigmoid,fit_list_sigma,fit_list_N_rho_mean/N,p0)   
        mresid = norm(fit.resid)/r # mean residuals
        p = fit.param
        #@show N,z,q,mresid,p
        p_err = LsqFit.standard_errors(fit)
        for i in 1:length(p)
            pi = p[i]
            pe = p_err[i]
        end
        #sigma_c          = p[2]
        #sigma_c_err      = p_err[2]
        #d2rho_peak       = dsigmoid(sigma_c,fit.param)
        #d2rho_peak_err   = estimate_error(dsigmoid,sigma_c,fit)
        A                = p[2]
        A_err            = p_err[2]
        B                = p[3]
        B_err            = p_err[3]
        sigma_c          = p[1]
        sigma_c_err      = p_err[1]
        d2rho_peak       = A*B/4 #dsigmoid(sigma_c,fit.param)
        d2rho_peak_err   = sqrt((B/4*A_err)^2+(A/4*B_err)^2) #estimate_error(dsigmoid,sigma_c,fit)        
        push!(list_sigma_c_vs_r,sigma_c)
        push!(list_sigma_c_err_vs_r,sigma_c_err)
        push!(list_d2rho_peak_vs_r,d2rho_peak)
        push!(list_d2rho_peak_err_vs_r,d2rho_peak_err)
        push!(list_mresid_vs_r,mresid)
        if d2rho_peak >= bestfit_d2rho_peak
            bestfit_r              = r
            bestfit_list_sigma     = fit_list_sigma
            bestfit_p              = p
            bestfit_sigma_c        = sigma_c
            bestfit_sigma_c_err    = sigma_c_err
            bestfit_d2rho_peak     = d2rho_peak
            bestfit_d2rho_peak_err = d2rho_peak_err
        end
    end      
    sigma5 = list_sigma[end]
    tau_at_sigma5 = list_N_tau_mean[end]/N
    rho_at_sigma5 = list_N_rho_mean[end]/N    
    @show N,k,bestfit_r,bestfit_sigma_c,bestfit_d2rho_peak,list_r,sigma5,tau_at_sigma5,rho_at_sigma5
    
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0,
        label="\$N=$N\$, \$k=$kk\$",
        markersize=2.0,
        markerstrokewidth=0.0,        
    )
    plot!(bestfit_list_sigma,x->psigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle",width=2.0,color=:black,style=:dash)
    plot!(bestfit_list_sigma,x->sigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle'",width=2.0)
    plot!(bestfit_list_sigma,x->dsigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle''",width=2.0)
    #scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\bar{\rho}\rangle''(\sigma_c)",markersize=4.0,markerstrokewidth=0.0)
    scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\tilde{\rho}\rangle",markersize=4.0,markerstrokewidth=0.0)
    scatter!([sigma_star],[0.0],label=L"\sigma^*",markersize=4.0,markerstrokewidth=0.0,c=:red,cl=:red,mc=:red)
    plot!(legend=:topright)
    
    #B = bbox(0.1,0.01,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    B = bbox(0.45,0.45,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    plot!(plt,inset = (1, B),subplot = 2)
    #plot!(xlabel=L"r",subplot=2)
    plot!(xlabel=L"\sigma^{**}",subplot=2)
    plot!(ylabel=L"\sigma_c",subplot=2,yguidefont = font(12,color=col6),)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],label="",subplot=2,c=col6,lc=col6,mc=col6)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    plot!(list_sigma[list_r[1:10:end]],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    #plot!(list_r,list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    plot!(list_sigma[list_r],list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    #scatter!([bestfit_r],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    scatter!([list_sigma[bestfit_r]],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    
    plt2 = twinx(plt[2])
    plot!(
        plt2,
        #xlabel=L"N",
        #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
        ylabel=L"\langle\tilde{\rho}\rangle",
        yguidefont = font(12,color=col7),
        #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
        #xscale=:log10,
        #yscale=:log10,
        subplot=2,
    )
    #scatter!(plt2,list_r[1:10:end],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    scatter!(plt2,list_sigma[list_r[1:10:end]],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    #plot!(plt2,list_r,list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    plot!(plt2,list_sigma[list_r],list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    
    ER_peaks[ky] = [plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star]    
    savefig("figs/SM-ER-sigma_c-N$N-k$kk.pdf")

end  
ER_peaks[(512,16.0)][1]

###### $\sigma_c$ vs $N$

In [ ]:
fig4b_1 = plot()
plot!(xscale=:log10)
#plot!(yscale=:log10)
#plot!(xticks=[1e2,1e3])
plot!(
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([1.0,1.1,1.2],[L"1.0",L"1.1",L"1.2"]),
    tickfont=font(11),
    xlim=(62,1050),
    ylim=(1.0,1.21),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)

list_N                   = []
list_sigma_c_vs_N        = []
list_sigma_c_err_vs_N    = []
list_d2rho_peak_vs_N     = []
list_d2rho_peak_err_vs_N = []
for ky in sort(collect(keys(ER_peaks)))
    (N,k) = ky    
    kk = Int(k)
    if kk != 16
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = ER_peaks[ky]
    push!(list_N,N)
    push!(list_sigma_c_vs_N,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_N,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_N,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_N,bestfit_d2rho_peak_err)
end
@show list_N,list_sigma_c_vs_N,list_d2rho_peak_vs_N

cola=:cyan
colb = :magenta

plot!(xlabel=L"N")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_N,list_sigma_c_vs_N,yerr=list_sigma_c_err_vs_N,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_N,list_sigma_c_vs_N,label="",c=col6,lc=col6,mc=col6)
@. fma(x,p) = p[1]*x^p[2]
imax=2
fit_fma = LsqFit.curve_fit(fma,list_N[imax:end],list_sigma_c_vs_N[imax:end],[1.0,1.0])
@show fit_fma.param
@show standard_errors(fit_fma)
a = fit_fma.param[1]
b = fit_fma.param[2]
err_a = standard_errors(fit_fma)[1]
err_b = standard_errors(fit_fma)[2]
c = b/a
err_c = sqrt(c^2*(err_a^2/a^2+err_b^2/b^2)) # err_c^2/c^2 = err_a^2/a^2+err_b^2/b^2
@show a,b
@show c,err_c
#plot!(list_N[imax:end],x->fma(x,fit_fma.param),c=cola,lc=cola,mc=cola,style=:dash,width=3.0,label=L"\sim (1.149\pm 0.007)-(0.007\pm 0.001)\ln N")
#plot!(list_N[imax:end],x->fma(x,fit_fma.param),c=cola,lc=cola,mc=cola,style=:dash,width=3.0,label=L"\sim (1-(0.006\pm 0.0009)\ln N)")
plot!(list_N[imax:end],x->fma(x,fit_fma.param),c=cola,lc=cola,mc=cola,style=:dash,width=3.0,label=L"\sim (1-10^{-3}(6\pm 0.9)\ln N)")

#plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (0.323\pm 0.003)+(0.02\pm 0.002)\ln N")
#plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (1+(0.06\pm 0.005)\ln N)")
plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (1+10^{-2}(6\pm 0.5)\ln N)")
fig4b_1_t = twinx(fig4b_1)
plot!(
    fig4b_1_t,
    #xlabel=L"N",
    #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    #yscale=:log10,
    ylim=(0.3,0.4),
)
plot!(fig4b_1_t,yticks=0.3:0.05:0.41)  
plot!(fig4b_1_t,list_N,list_d2rho_peak_vs_N,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4b_1_t,list_N,list_d2rho_peak_vs_N,yerr=list_d2rho_peak_err_vs_N,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([0.30,0.35,0.40],[L"0.30",L"0.35",L"0.40"]),
    tickfont=font(11),    
    xlim=(62,1050),
)
@. fmb(x,p) = p[1]*x^p[2]
imax=2
fit_fmb = LsqFit.curve_fit(fmb,list_N[imax:end],list_d2rho_peak_vs_N[imax:end],[1.0,1.0])
@show fit_fmb.param
@show standard_errors(fit_fmb)
a = fit_fmb.param[1]
b = fit_fmb.param[2]
err_a = standard_errors(fit_fmb)[1]
err_b = standard_errors(fit_fmb)[2]
c = b/a
err_c = sqrt(c^2*(err_a^2/a^2+err_b^2/b^2)) # err_c^2/c^2 = err_a^2/a^2+err_b^2/b^2
@show a,b
@show c,err_c
plot!(subplot=2,list_N[imax:end],x->fmb(x,fit_fmb.param),c=colb,lc=colb,mc=colb,style=:dash,width=3.0,label="")

#savefig("figs/tmp-ER-sigma_c-vs-N.pdf")
plot!()

###### $\sigma_c$ vs $\bar{k}$

In [ ]:
fig4b_2 = plot()
#plot!(xlim=(0.0,5.5))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
#plot!(xscale=:log10)
plot!(
    xticks=([4,8,16],[L"2^2",L"2^3",L"2^4"]),
    yticks=([2^-3,2^-2,2^-1,2^0,2^1,2^2],[L"2^{-3}",L"2^{-2}",L"2^{-1}",L"2^{0}",L"2^{1}",L"2^{2}"]),
    ylims=(2^-2,2^1),
    tickfont=font(11),
)
plot!(legend=:top,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)

list_k                   = []
list_sigma_c_vs_k        = []
list_sigma_c_err_vs_k    = []
list_d2rho_peak_vs_k     = []
list_d2rho_peak_err_vs_k = []
for ky in sort(collect(keys(ER_peaks)))
    (N,k) = ky    
    kk = Int(k)
    if N != 512
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = ER_peaks[ky]
    push!(list_k,kk)
    push!(list_sigma_c_vs_k,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_k,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_k,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_k,bestfit_d2rho_peak_err)
end
#@show list_k,list_sigma_c_vs_k,list_d2rho_peak_vs_k

cola=:cyan
colb = :magenta

@. fma(x,p) = p[1]*x^p[2]*log(x)
fit_fma = LsqFit.curve_fit(fma,list_k[1:end],list_sigma_c_vs_k[1:end],[0.5,0.5])
@show fit_fma.param
@show standard_errors(fit_fma)
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(xlabel=L"\bar{k}")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_k[1:end],x->fma(x,fit_fma.param),label=L"\sim \ln(\bar{k}) \times \bar{k}^{0.193\pm 0.003}",c=cola,lc=cola,mc=cola,style=:dash,width=3.0)
plot!(list_k,list_sigma_c_vs_k,yerr=list_sigma_c_err_vs_k,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_k,list_sigma_c_vs_k,label="",c=col6,lc=col6,mc=col6)

plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim \bar{k}^{-1.080\pm 0.003}")
@. fmb(x,p) = p[1]*x^p[2]#*log(x)
fit_kmin = 4
fit_fmb = LsqFit.curve_fit(fmb,list_k[fit_kmin:end],list_d2rho_peak_vs_k[fit_kmin:end],[2.0,-1.0])
@show fit_fmb.param
@show standard_errors(fit_fmb)
fig4b_2_t = twinx(fig4b_2)
plot!(
    fig4b_2_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    yscale=:log10,
)
plot!(   fig4b_2_t,list_k[fit_kmin:end],x->fmb(x,fit_fmb.param),label="",c=colb,lc=colb,mc=colb,style=:dash,width=3.0)
plot!(   fig4b_2_t,list_k,list_d2rho_peak_vs_k,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4b_2_t,list_k,list_d2rho_peak_vs_k,yerr=list_d2rho_peak_err_vs_k,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    xticks=([1,2,4,8,16],[L"2^0",L"2^1",L"2^2",L"2^3",L"2^4"]),
    yticks=([2^-3,2^-2,2^-1,2^0,2^1,2^2],[L"2^{-3}",L"2^{-2}",L"2^{-1}",L"2^{0}",L"2^{1}",L"2^{2}"]),
    ylims=(2^-2,2^2),
    tickfont=font(11),
)

#savefig("figs/tmp-ER-sigma_c-vs-k.pdf")
plot!()

##### $|f|$ varying $N$

In [ ]:
figSM9b_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
#plot!(ylim=(1.15,1.191))
plot!(ylim=(1.1,1.21))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-3/2}|f|")
#plot!(yticks=1.15:0.01:1.19)
#plot!(yticks=1.1:0.05:1.2)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([1.1,1.15,1.2],[L"1.10",L"1.15",L"1.20"]),
    tickfont=font(11),
)
colors_N = palette(cols1,5)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k != 16.0
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_f_mean/N^1.5,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,             
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

##### $|f|$ varying $k$

In [ ]:
figSM9b_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e3,1e5))
#plot!(ylim=(0.27,0.301))
plot!(ylim=(0.2,0.31))
#plot!(yticks=(0.27:0.01:0.3))
#plot!(yticks=(0.2:0.05:0.3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"k^{-1/2}N^{-3/2}|f|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0.2,0.25,0.3],[L"0.20",L"0.25",L"0.30"]),
    tickfont=font(11),
)
colors_k = palette(cols2,13)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N != 512
        continue
    end
    c+=1
    kk=Int(k)
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_f_mean/k^0.5/N^1.5,#/N^1.5,
        #yerr=list_rho_r_n_std/N,
        label="\$k=$kk\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        c=colors_k[c],        
    )
end  
#scatter!(list_sigma,list_norm_f/N,yerr=list_norm_f/N,label="")
plot!()

##### $|s|$ varying $N$

In [ ]:
figSM10b_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^{-1}|s|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([2,4,6,8,10,12],[L"2",L"4",L"6",L"8",L"10",L"12"]),
    tickfont=font(11),
)
colors_N = palette(cols1,5)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k != 16.0
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_s_mean/10,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        c=colors_N[c],           
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!(x->2.6*x+0.5,c=:black,style=:dash,label=L"2.6\sigma+0.5",width=3.0)
plot!()

##### $|s|$ varying $k$

In [ ]:
figSM10b_2 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"k^{-3/2}|s|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2],[L"0",L"1",L"2"]),
    tickfont=font(11),
)
colors_k = palette(cols2,13)
colors_k_aux = Vector{Any}(undef,length(colors_k))
colors_k_aux .= colors_k
colors_k_aux[end] = :red
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N != 512
        continue
    end
    c+=1
    kk=Int(k)
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_s_mean/k^1.5,
        #yerr=list_rho_r_n_std/N,
        label="\$k=$kk\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        #c=colors_k[c],
        c=colors_k_aux[c],
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!(x->0.41*x+0.1,c=:black,style=:dash,width=3.0,label=L"0.41\sigma+0.1")
plot!()

##### $|h|$ varying $N$

In [ ]:
figSM11b_1 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^{-2}|h|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
list_N = []
list_norm_h_mean_sigma_end = []
colors_N = palette(cols1,5)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k != 16.0
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_h_mean_sigma_end,list_norm_h_mean[end])
    scatter!(
        list_sigma,
        list_norm_h_mean/100,
        #list_norm_h_mean/N^0.55,
        #list_norm_h_mean,#/N^1.3/(log(N))^1.5,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        c=colors_N[c],
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")

B = bbox(0.23,0.05,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
plot!(
    subplot=2,
    #xticks=([64,128,256,512,1024],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    #yticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),
    #yticks=([0,2,4,6],[L"0",L"2",L"4",L"6"]),
    yticks=([0,2,4],[L"0",L"2",L"4"]),
    ylim=(0,4.2),
    tickfont=font(11),
)
p = [1.0,1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)
#p = [1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*log(x)
fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    xlabel=L"N",
    ylabel=L"10^{-2}|h|(\sigma=5)",
    list_N,
    list_norm_h_mean_sigma_end/100,
    label="",
    xscale=:log10,
    #yscale=:log10,
    subplot=2,   
    #xticks=([64,128,256,512,1024],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}"]),
    #legend=:topleft,
    legend=:top,
)
plot!(
    64:1.0:1024,
    x->fm(x,p)/100,
    #label=L"(-1.70\pm 0.1)+(0.17\pm 0.02)N^{0.23\pm 0.01}\ln N",
    label="",
    subplot=2,    
)
#plot!()

In [ ]:
p = [1.0,1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)

#p = [1.0,1.0]
#@. fm(x,p) = p[1]+x^p[2]*log(x)

#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]*log(x)

fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_h_mean_sigma_end,p)

@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    xlabel=L"N",
    ylabel=L"|h|",
    list_N,
    list_norm_h_mean_sigma_end,
    label="",
    xscale=:log10,
    #yscale=:log10,
)
plot!(x->fm(x,p),label=L"...")

##### $|h|$ varying $k$

In [ ]:
figSM11b_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(-1.0,32.0))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^{-2}|h|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    #yticks=([0,10,20,30],[L"0",L"10",L"20",L"30"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    ylim=(0,3),
    tickfont=font(11),
)
#plot!(ylabel=L"|h|\,k^{-0.12}\,\ln^{-3/5}(k)")
colors_k = palette(cols2,13)
colors_k_aux = vcat([c for c in colors_k[1:end-1]],[:red])
list_k = []
list_norm_h_mean_sigma_end = []
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N != 512
        continue
    end
    c+=1
    kk=Int(k)
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    push!(list_k,k)
    push!(list_norm_h_mean_sigma_end,list_norm_h_mean[end])
    scatter!(
        list_sigma,        
        list_norm_h_mean/100,
        #list_norm_h_mean/k^0.5,
        #list_norm_h_mean/k^-0.12/(log(k))^1.5,        
        #yerr=list_rho_r_n_std/N,
        label="\$\\bar{k}=$kk\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        #c=colors_k[c],
        c=colors_k_aux[c],
    )
end  
plot!()
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")

B = bbox(0.2,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
p = [1.0,1.0]
@. fm(x,p) = p[1]+p[2]*log(x)
fit_fm = LsqFit.curve_fit(fm,list_k,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    label="",
    xscale=:log10,
    #yscale=:log10,
    list_k,
    list_norm_h_mean_sigma_end/100,
    xlabel=L"\bar{k}",
    ylabel=L"10^{-2}|h|(\sigma=5)",
    #xticks=[10,100],
    subplot=2,
    yticks=[10,15,20,25],
)
plot!(
    subplot=2,
    #xticks=([4,8,16],[L"2^2",L"2^3",L"2^4"]),
    xticks=([4,8],[L"2^2",L"2^3"]),
    #yticks=([15,20,25],[L"15",L"20",L"25"]),
    yticks=([1,2,3],[L"1",L"2",L"3"]),
    ylim=(1,3),
    tickfont=font(11),
)
plot!(
    4.0:1.0:16.0,
    x->fm(x,p)/100,
    #label=L"(-4.7\pm 0.1)+(11.46\pm 0.05)\ln \bar{k}",
    label="",
    subplot=2,    
)

In [ ]:
@show list_k
@show list_norm_h_mean_sigma_end

p = [1.0,1.0]
@. fm(x,p) = p[1]+p[2]*log(x)

#p = [1.0,1.0]
#@. fm(x,p) = p[1]+x^p[2]*log(x)

#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]*log(x)

fit_fm = LsqFit.curve_fit(fm,list_k,list_norm_h_mean_sigma_end,p)

@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    label="",
    xscale=:log10,
    #yscale=:log10,
    list_k,
    list_norm_h_mean_sigma_end,
    #list_norm_h_mean_sigma_end/10,
    xlabel=L"\bar{k}",
    ylabel=L"|h|(\sigma=5)",
    #ylabel=L"10^{-1}|h|(\sigma=5)",    
    xticks=([1,2,4,8,16],[L"2^0",L"2^1",L"2^2",L"2^3",L"2^4"])
)
plot!(
    x->fm(x,p),    
    #x->fm(x,p)/10,
    label=L"(-47\pm 1)+(114.6\pm 0.5)\ln \bar{k}",
)

##### $|g|$ varying $N$

In [ ]:
figSM12b_1 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-3/2}|g|")
#plot!(ylim=(1.15,1.171))
#plot!(yticks=1.15:0.01:1.17)
plot!(ylim=(1.1,1.2))
#plot!(yticks=1.1:0.05:1.21)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([1.1,1.15,1.20],[L"1.10",L"1.15",L"1.20"]),
    tickfont=font(11),
)

colors_N = palette(cols1,5)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k != 16.0
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_g_mean/N^1.5,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        c=colors_N[c],
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

##### $|g|$ varying $k$

In [ ]:
figSM12b_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"k^{-1/2}N^{-3/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([26,28,30],[L"26",L"28",L"30"]),
    tickfont=font(11),
)
#plot!(ylim=(0.280,0.291))
#plot!(yticks=0.280:0.01:0.290)
#plot!(ylim=(0.2,0.3))
plot!(ylim=(26,30))
#plot!(yticks=0.2:0.05:0.3)

colors_k = palette(cols2,13)
c=0
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N != 512
        continue
    end
    c+=1
    kk=Int(k)
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        #list_norm_g_mean,#/(log(k))^1.0,#/N^1.3/(log(N))^1.5,
        list_norm_g_mean/N^1.5/k^0.5*100,
        #yerr=list_rho_r_n_std/N,
        label="\$\\bar{k}=$kk\$",
        markersize=3.0,
        markerstrokewidth=0.0,         
        c=colors_k[c],
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

##### $\kappa_k$ vs $N$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13b_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{ER},\;\bar{k}=16",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"N")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
plot!(
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([10^2,10^3],[L"10^2",L"10^3"]),
    tickfont=font(11),
)
list_N = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if k == 16.0
        push!(list_N,N)
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

#plot!(128.0:128.0,x->128.0,label="\$\\bar{k}=16\$",width=0.0,c=:white)

scatter!(
    list_N,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_N,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_N,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2,
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_N,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_N,
    #list_m_1_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_1_std,
    label=L"\langle \kappa_1\rangle\propto N",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,    
    c=2,
)

scatter!(
    list_N,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_N,list_m_2_mean,[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_N,
    #list_m_2_mean,
    x->fm2(x,fit_m2.param),
    #yerr=list_m_2_std,
    label=L"\langle \kappa_2\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,      
    c=3
)

plot!()

##### $\kappa_k$ varying $k$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13b_2 = plot()
plot!(legend=:right,legendtitle=L"\mathrm{ER},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\bar{k}")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(
    xticks=([2^2,2^3,2^4],[L"2^2",L"2^3",L"2^4"]),
    yticks=([10^2,10^3],[L"10^2",L"10^3"]),
    tickfont=font(11),
)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
list_k = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(ER_stats_vs_sigma)))
    (N,k) = ky
    if N == 512
        push!(list_k,k)
        list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = ER_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

#plot!(8.0:8.0,x->8.0,label=L"N=512",width=0.0,c=:white)

scatter!(
    list_k,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1,
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_k,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_k,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,      
    c=1,
)

scatter!(
    list_k,
    list_m_1_mean,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2,
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_k,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_k,
    #list_m_1_mean,
    x->fm1(x,fit_m1.param),    
    #yerr=list_m_1_std,
    label=L"\langle \kappa_1\rangle\propto \bar{k}",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,       
    c=2,
)

scatter!(
    list_k,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_k,list_m_2_mean,[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_k,
    #list_m_2_mean,
    x->fm2(x,fit_m2.param),        
    #yerr=list_m_2_std,
    label=L"\langle \kappa_2\rangle\propto \bar{k}^3",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,    
    c=3,
)
plot!(legend=:bottomright)

### Barabasi-Albert

#### Developing example

In [ ]:
N = 1000
m = 3
@show N
#@show mean_k
#@show p
g = barabasi_albert(N, m)
@show nv(g)
@show ne(g)
@show 2*ne(g)/N
#@show g
;

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="BA")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="BA")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

#### Disorder parameter vs noise

In [ ]:
filename = "data/BA_stats_vs_sigma.jld2"

# Deserialize data
if isfile(filename)
    @load filename BA_stats_vs_sigma
else    
    BA_stats_vs_sigma = Dict()    
end

##### $\bar{\tau}$ varying $N$

In [ ]:
fig2c_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\tau}\rangle")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"m=3",width=0.0,c=:white)
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,m) = ky
    if m == 3
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_tau_mean/N,
            #yerr=list_rho_w_n_std/N,
            label="\$N=$N\$",#  \$m=$m\$",
            c=colors_N[c],
            markersize=3.0,
            markerstrokewidth=0.0,            
        )
    end
end    
plot!()
#savefig("figs/fig3a.pdf")

##### $\bar{\rho}$ varying $N$

In [ ]:
fig3c_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"m=3",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,m) = ky
    if m == 3
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_rho_mean/N,
            #yerr=list_N_rho_std/N,
            label="\$N=$N\$",#  \$m=$m\$"
            c=colors_N[c],
            markersize=3.0,
            markerstrokewidth=0.0,              
        )
    end
end    
plot!()

##### $\bar{\tau}$ varying $q$

In [ ]:
fig2c_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"q^{2/3}\langle\bar{\tau}\rangle")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,2,4,6,8,10,12],[L"0",L"2",L"4",L"6",L"8",L"10",L"12"]),
    tickfont=font(11),
)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"N=512",width=0.0,c=:white)
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
colors_q = palette(cols2,8)
colors_q_aux = vcat([c for c in colors_q[1:7]],[:red])
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N == 512
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_tau_mean*q^(2/3)/N,
            #yerr=list_N_tau_std/N,
            #label="\$N=$N\$  \$m=$m\$"
            label="\$q=$q\$",
            #c=colors_m[c],
            c=colors_q_aux[c],
            markersize=3.0,
            markerstrokewidth=0.0,             
        )
    end
end    
plot!()

##### $\bar{\rho}$ varying $q$

In [ ]:
fig3c_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"N=512",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
colors_q = palette(cols2,8)
colors_q_aux = vcat([c for c in colors_q[1:7]],[:red])
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N == 512
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_rho_mean/N,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$m=$m\$"
            label="\$q=$q\$",
            #c=colors_m[c],
            c=colors_q_aux[c],
            markersize=3.0,
            markerstrokewidth=0.0,
        )
    end
end    
plot!()

##### $\sigma_c$

In [ ]:
BA_peaks = Dict()

for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    #if k != 1.0
    #    continue
    #end
    #@show ky
    
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
    
    #@show list_sigma
    #@show list_N_rho
    #@show list_N_rho_mean
   
    plt = plot()
    plot!(legend=:topleft,legendtitle=L"\mathrm{BA}",legendtitlefontsize=8)
    #plot!(xlim=(0.0,2.0))
    #plot!(ylim=(-0.5,0.5))
    #plot!(ylim=(-0.5,:auto))
    plot!(xlabel=L"\sigma")
    plot!(ylabel=L"\langle\bar{\rho}\rangle")    

    r_star = find_first_nonzero(list_N_rho_mean)-1
    sigma_star = list_sigma[r_star]
    
    if q == 1
        list_r = 15:80
    else
        list_r = 2*r_star:200
    end
    #list_r = 3*q_star:300
    
    list_sigma_c_vs_r        = []
    list_sigma_c_err_vs_r    = []
    list_d2rho_peak_vs_r     = []
    list_d2rho_peak_err_vs_r = []
    list_mresid_vs_r         = []
    bestfit_r                = 0
    bestfit_list_sigma       = 0
    bestfit_p                = 0
    bestfit_sigma_c          = 0
    bestfit_sigma_c_err      = 0
    bestfit_d2rho_peak       = 0
    bestfit_d2rho_peak_err   = 0

    #p = [10.0,1.3,0.25,0.0]
    p = [1.3,0.25,10.0,0.0]    
    for r in list_r
        p0 = p
        fit_range = r_star:r
        fit_list_sigma = collect(list_sigma[fit_range])
        fit_list_N_rho_mean = list_N_rho_mean[fit_range]
        fit = LsqFit.curve_fit(psigmoid,fit_list_sigma,fit_list_N_rho_mean/N,p0)   
        mresid = norm(fit.resid)/r # mean residuals
        p = fit.param
        #@show N,z,q,mresid,p
        p_err = LsqFit.standard_errors(fit)
        for i in 1:length(p)
            pi = p[i]
            pe = p_err[i]
        end
        #sigma_c          = p[2]
        #sigma_c_err      = p_err[2]
        #d2rho_peak       = dsigmoid(sigma_c,fit.param)
        #d2rho_peak_err   = estimate_error(dsigmoid,sigma_c,fit)
        A                = p[2]
        A_err            = p_err[2]
        B                = p[3]
        B_err            = p_err[3]
        sigma_c          = p[1]
        sigma_c_err      = p_err[1]
        d2rho_peak       = A*B/4 #dsigmoid(sigma_c,fit.param)
        d2rho_peak_err   = sqrt((B/4*A_err)^2+(A/4*B_err)^2) #estimate_error(dsigmoid,sigma_c,fit)                
        push!(list_sigma_c_vs_r,sigma_c)
        push!(list_sigma_c_err_vs_r,sigma_c_err)
        push!(list_d2rho_peak_vs_r,d2rho_peak)
        push!(list_d2rho_peak_err_vs_r,d2rho_peak_err)
        push!(list_mresid_vs_r,mresid)
        if d2rho_peak >= bestfit_d2rho_peak
            bestfit_r              = r
            bestfit_list_sigma     = fit_list_sigma
            bestfit_p              = p
            bestfit_sigma_c        = sigma_c
            bestfit_sigma_c_err    = sigma_c_err
            bestfit_d2rho_peak     = d2rho_peak
            bestfit_d2rho_peak_err = d2rho_peak_err
        end
    end      
    sigma5 = list_sigma[end]
    tau_at_sigma5 = list_N_tau_mean[end]/N
    rho_at_sigma5 = list_N_rho_mean[end]/N
    @show N,q,bestfit_r,bestfit_sigma_c,bestfit_d2rho_peak,list_r,sigma5,tau_at_sigma5,rho_at_sigma5
    
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0,
        label="\$N=$N\$, \$q=$q\$",
        markersize=2.0,
        markerstrokewidth=0.0,        
    )
    plot!(bestfit_list_sigma,x->psigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle",width=2.0,color=:black,style=:dash)
    plot!(bestfit_list_sigma,x->sigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle'",width=2.0)
    plot!(bestfit_list_sigma,x->dsigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle''",width=2.0)
    #scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\bar{\rho}\rangle''(\sigma_c)",markersize=4.0,markerstrokewidth=0.0)
    scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\tilde{\rho}\rangle",markersize=4.0,markerstrokewidth=0.0)
    scatter!([sigma_star],[0.0],label=L"\sigma^*",markersize=4.0,markerstrokewidth=0.0,c=:red,cl=:red,mc=:red)
    plot!(legend=:topright)
    
    #B = bbox(0.1,0.01,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    B = bbox(0.45,0.45,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    plot!(plt,inset = (1, B),subplot = 2)
    #plot!(xlabel=L"r",subplot=2)
    plot!(xlabel=L"\sigma^{**}",subplot=2)
    plot!(ylabel=L"\sigma_c",subplot=2,yguidefont = font(12,color=col6),)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],label="",subplot=2,c=col6,lc=col6,mc=col6)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)    
    plot!(list_sigma[list_r[1:10:end]],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)    
    #plot!(list_r,list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    plot!(list_sigma[list_r],list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    #scatter!([bestfit_r],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    scatter!([list_sigma[bestfit_r]],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    
    plt2 = twinx(plt[2])
    plot!(
        plt2,
        #xlabel=L"N",
        #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
        ylabel=L"\langle\tilde{\rho}\rangle",
        yguidefont = font(12,color=col7),
        #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
        #xscale=:log10,
        #yscale=:log10,
        subplot=2,
    )
    #scatter!(plt2,list_r[1:10:end],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    scatter!(plt2,list_sigma[list_r[1:10:end]],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    #plot!(plt2,list_r,list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    plot!(plt2,list_sigma[list_r],list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    
    BA_peaks[ky] = [plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star]    
    savefig("figs/SM-BA-sigma_c-N$N-q$q.pdf")

end  
BA_peaks[(512,3)][1]

###### $\sigma_c$ vs $N$

In [ ]:
fig4c_1 = plot()
#plot!(xlim=(0.0,5.5))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
plot!(xscale=:log10)
plot!(legend=:topright,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
plot!(
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([0.5,0.55,0.6],[L"0.5",L"0.55",L"0.6",L"6"]),
    tickfont=font(11),
    ylims=(0.5,0.6),
)

list_N                   = []
list_sigma_c_vs_N        = []
list_sigma_c_err_vs_N    = []
list_d2rho_peak_vs_N     = []
list_d2rho_peak_err_vs_N = []
for ky in sort(collect(keys(BA_peaks)))
    (N,q) = ky    
    if q != 3
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = BA_peaks[ky]
    push!(list_N,N)
    push!(list_sigma_c_vs_N,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_N,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_N,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_N,bestfit_d2rho_peak_err)
end
@show list_N,list_sigma_c_vs_N,list_d2rho_peak_vs_N

cola = :cyan
colb = :magenta

#plot!(ylims=(0.5,0.61))
#plot!(yticks=0.5:0.05:0.6)
plot!(xlabel=L"N")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_N,list_sigma_c_vs_N,yerr=list_sigma_c_err_vs_N,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_N,list_sigma_c_vs_N,label="",c=col6,lc=col6,mc=col6)
#@. fma(x,p) = p[1]*x^p[2]
#imax=3
#fit_fma = LsqFit.curve_fit(fma,list_N[imax:end],list_sigma_c_vs_N[imax:end],[1.0,1.0])
#@show fit_fma.param
#@show standard_errors(fit_fma)
#plot!(list_N[imax:end],x->fma(x,fit_fma.param),label="",c=cola,lc=cola,mc=cola,style=:dash,width=3.0)

#plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (0.323\pm 0.003)+(0.02\pm 0.002)\ln N")
#plot!(100.0:0.1:100.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (1+(0.006\pm 0.0001)\ln N)")
plot!(100.0:0.1:100.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim (1+10^{-3}(6\pm 0.1)\ln N)")
fig4c_1_t = twinx(fig4c_1)
plot!(
    fig4c_1_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    #ylims=(1.0,1.5),
    #yscale=:log10,
)
plot!(fig4c_1_t,list_N,list_d2rho_peak_vs_N,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4c_1_t,list_N,list_d2rho_peak_vs_N,yerr=list_d2rho_peak_err_vs_N,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([1.0,1.1,1.2,1.3,1.4,1.5],[L"1.0",L"1.1",L"1.2",L"1.3",L"1.4",L"1.5"]),
    tickfont=font(11),
    ylims=(1.0,1.5),
)
@. fmb(x,p) = p[1]*x^p[2]
imax=3
fit_fmb = LsqFit.curve_fit(fma,list_N[imax:end],list_d2rho_peak_vs_N[imax:end],[1.0,1.0])
@show fit_fmb.param
@show standard_errors(fit_fmb)
a = fit_fmb.param[1]
b = fit_fmb.param[2]
err_a = standard_errors(fit_fmb)[1]
err_b = standard_errors(fit_fmb)[2]
c = b/a
err_c = sqrt(c^2*(err_a^2/a^2+err_b^2/b^2)) # err_c^2/c^2 = err_a^2/a^2+err_b^2/b^2
@show a,b
@show c,err_c
plot!(subplot=2,list_N[imax:end],x->fmb(x,fit_fmb.param),label="",c=colb,lc=colb,mc=colb,style=:dash,width=3.0)

#savefig("figs/tmp-ER-sigma_c-vs-N.pdf")
plot!()

###### $\sigma_c$ vs $q$

In [ ]:
fig4c_2 = plot()
#plot!(xlim=(0.0,5.5))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(
    #xticks=([2^0,2^1,2^2,2^3,2^4,2^5,2^6,2^7,2^8],[L"2^0",L"2^1",L"2^2",L"2^3",L"2^4",L"2^5",L"2^6",L"2^7",L"2^8"]),
    xticks=([2^0,2^1,2^2,2^3],[L"2^0",L"2^1",L"2^2",L"2^3"]),
    yticks=([2^-3,2^-2,2^-1,2^0],[L"2^{-3}",L"2^{-2}",L"2^{-1}",L"2^{-0}"]),
    tickfont=font(11),
    ylims=(2^-3,2^0),
)
plot!(legend=:bottom,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)

list_q                   = []
list_sigma_c_vs_q        = []
list_sigma_c_err_vs_q    = []
list_d2rho_peak_vs_q     = []
list_d2rho_peak_err_vs_q = []
for ky in sort(collect(keys(BA_peaks)))
    (N,q) = ky    
    if N != 512
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = BA_peaks[ky]
    push!(list_q,q)
    push!(list_sigma_c_vs_q,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_q,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_q,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_q,bestfit_d2rho_peak_err)
end
#@show list_m,list_sigma_c_vs_m,list_d2rho_peak_vs_m

cola = :cyan
colb = :magenta

@. fma(x,p) = p[1]*x^p[2]
fit_fma = LsqFit.curve_fit(fma,list_q[2:end],list_sigma_c_vs_q[2:end],[0.3,0.6])
@show fit_fma.param
@show standard_errors(fit_fma)

#plot!(xlims=(0,:auto))
plot!(xlabel=L"q")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_q[2:end],x->fma(x,fit_fma.param),label=L"\sim q^{0.587\pm 0.008}",c=cola,lc=cola,mc=cola,style=:dash,width=3.0)
plot!(list_q,list_sigma_c_vs_q,yerr=list_sigma_c_err_vs_q,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_q,list_sigma_c_vs_q,label="",c=col6,lc=col6,mc=col6)

plot!(9.0:0.1:9.0,x->1.0,c=colb,style=:dash,width=3.0,label=L"\sim q^{-1.27\pm 0.03}")

@. fmb(x,p) = p[1]*x^p[2]
fit_fmb = LsqFit.curve_fit(fmb,list_q[2:end],list_d2rho_peak_vs_q[2:end],[5.7,-1.3])
@show fit_fmb.param
@show standard_errors(fit_fmb)

fig4c_2_t = twinx(fig4c_2)
plot!(
    fig4c_2_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    yscale=:log10,
)
#plot!(   fig4c_2_t,yscale=:log10)  
plot!(   fig4c_2_t,list_q[2:end],x->fmb(x,fit_fmb.param),label="",c=colb,lc=colb,mc=colb,style=:dash,width=3.0)
plot!(   fig4c_2_t,list_q,list_d2rho_peak_vs_q,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4c_2_t,list_q,list_d2rho_peak_vs_q,yerr=list_d2rho_peak_err_vs_q,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    #xticks=([2^0,2^1,2^2,2^3,2^4,2^5,2^6,2^7,2^8],[L"2^0",L"2^1",L"2^2",L"2^3",L"2^4",L"2^5",L"2^6",L"2^7",L"2^8"]),
    xticks=([2^0,2^1,2^2,2^3],[L"2^0",L"2^1",L"2^2",L"2^3"]),
    yticks=([2^-1,2^0,2^1,2^2,2^3,2^4],[L"2^{-1}",L"2^{0}",L"2^{1}",L"2^{2}",L"2^{3}",L"2^{4}"]),
    tickfont=font(11),
    #ylims=(2^-3,2^0),
)
#savefig("figs/tmp-ER-sigma_c-vs-k.pdf")
plot!()

##### $|f|$ varying $N$

In [ ]:
figSM9c_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.68,0.731))
#plot!(yticks=0.68:0.01:0.73)
#plot!(ylim=(0.5,1.0))
#plot!(yticks=0.68:0.01:0.73)
plot!(
    xticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([6,7,8],[L"6",L"7",L"8"]),
    tickfont=font(11),
    ylims=(6,8),
)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}|f|")
list_N = []
list_norm_f_mean_at_sigma0 = []
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if q != 3
        continue
    end            
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_f_mean_at_sigma0,list_norm_f_mean[1]/N^1.5)            
    scatter!(
        list_sigma,
        list_norm_f_mean/N^1.5*10,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,        
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
#p = [0.72,1.5,1e-3,1.0]
#@. fm(x,p) = p[1]-p[2]*exp(-p[3]*x)*x^(-p[4])

#fit_p = [0.72,1e-3,1.0]
#@. fm(x,p) = p[1]*(1-exp(-p[2]*x)*x^(-p[3]))

p = [0.72,1.5,1.0]
@. fm(x,p) = p[1]-p[2]*x^(-p[3])

fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_f_mean_at_sigma0,p)
@show fit_fm.param
p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(list_N,list_norm_f_mean_at_sigma0,label="")
#plot!(list_N,x->fm(x,p),label=L"(0.7290\pm 0.0007)-(1.6\pm 0.2)N^{-(0.85\pm 0.03)}e^{-(0.0005\pm 0.0003)N}")
plot!(list_N,x->fm(x,p),label=L"(0.7305\pm 0.0003)-(1.7\pm 0.1)N^{-(0.86\pm 0.02)}")
plot!(
  yscale=:log10,
  xscale=:log10,
  xlabel=L"N",
  ylabel=L"N^{-3/2}|f|(\sigma=0)",
)

##### $|f|$ varying $q$

In [ ]:
figSM9c_2 = plot()
plot!(legend=:topright,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.4,0.431))
#plot!(yticks=0.4:0.01:0.43)
#plot!(ylim=(0.4,0.5))
#plot!(yticks=0.4:0.05:0.5)
plot!(
    xticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),
    yticks=([40,42,44,46],[L"40",L"42",L"44",L"46"]),
    tickfont=font(11),
    ylims=(40,46),
)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"100 q^{-1/2}N^{-3/2}|f|")
list_q = []
list_norm_f_mean_at_sigma0 = []
colors_q = palette(cols2,8)
colors_q_aux = Vector{Any}(undef,length(colors_q))
colors_q_aux .= colors_q
colors_q_aux[end] = :red
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    push!(list_q,q)
    push!(list_norm_f_mean_at_sigma0,list_norm_f_mean[1]/q^0.5/N^1.5)
    scatter!(
        list_sigma,
        list_norm_f_mean/q^0.5/N^1.5*100,
        #yerr=list_rho_r_n_std/N,
        label="\$q=$q\$",
        #c=colors_m[c],
        c=colors_q_aux[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
#p = [0.72,1.5,1e-3,1.0]
#@. fm(x,p) = p[1]-p[2]*exp(-p[3]*x)*x^(-p[4])

#fit_p = [0.72,1e-3,1.0]
#@. fm(x,p) = p[1]*(1-exp(-p[2]*x)*x^(-p[3]))

#p = [0.72,1.5,1.0]
#@. fm(x,p) = p[1]-p[2]*x^(-p[3])

#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^(-p[2])

p = [0.5,-0.001]
@. fm(x,p) = p[1]+p[2]*x


fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_f_mean_at_sigma0,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(list_q,list_norm_f_mean_at_sigma0,label="")
plot!(list_q,x->fm(x,p),label=L"(0.4219\pm 0.0001)-(0.00144\pm 0.00003)m")
plot!(
#  yscale=:log10,
#  xscale=:log10,
  xlabel=L"q",
  ylabel=L"q^{-1/2}|f|(\sigma=0)",
)

##### $|s|$ varying $N$

In [ ]:
figSM10c_1 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^{-1}|s|")
plot!(
    xticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),
    #yticks=([0,1,2,3,4,5,6,7,8],[L"0",L"1",L"2",L"3",L"4",L"5",L"6",L"7",L"8"]),
    yticks=([0,2,4,6,8],[L"0",L"2",L"4",L"6",L"8"]),
    tickfont=font(11),
    #ylims=(40,46),
)
#plot!(ylabel=L"N^{-0.05}|s|/\ln\,N")
list_N = []
list_norm_s_mean_sigma_end = []
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if q != 3
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_s_mean_sigma_end,list_norm_s_mean[end])
    #plot!(
    scatter!(
        list_sigma,
        list_norm_s_mean/10,
        #list_norm_s_mean/N^0.175,
        #list_norm_s_mean/N^0.05/log(N),
        #list_norm_s_mean/(M*log(M)),
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,             
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
#savefig("figs/tmp.pdf")

B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
p = [1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*x
@. fm(x,p) = p[1]+p[2]*log(x)
fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_s_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    subplot=2,
    list_N,
    list_norm_s_mean_sigma_end/10,
    xlabel=L"N",
    ylabel=L"10^{-1}|s|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    subplot=2,
    list_N[1]:1:list_N[end],
    x->fm(x,p)/10,
    #label=L"(-1.9\pm 0.1)+(1.35\pm 0.02)\ln N",
    label="",
)
plot!(
    subplot=2,
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    #yticks=([0,1,2,3,4,5,6,7,8],[L"0",L"1",L"2",L"3",L"4",L"5",L"6",L"7",L"8"]),
    yticks=([4,6,8],[L"4",L"6",L"8"]),
    tickfont=font(11),
    #ylims=(40,46),
)

In [ ]:
p = [1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*x
@. fm(x,p) = p[1]+p[2]*log(x)

fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_s_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_N,
    list_norm_s_mean_sigma_end,
    xlabel=L"N",
    ylabel=L"|s|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    list_N[1]:1:list_N[end],
    x->fm(x,p),
    label=L"(-19\pm 1)+(13.5\pm 0.2)\ln N",
)

In [ ]:
a = -1.9
b = 1.35
N0 = exp(-a/b)

##### $|s|$ varying $q$

In [ ]:
figSM10c_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-0,1e2))
plot!(xlabel=L"\sigma")
#plot!(ylabel=L"m^{-0.6}|s|/\ln(m)")
plot!(ylabel=L"10^{-2}|s|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2],[L"0",L"1",L"2"]),
    tickfont=font(11),
    #ylims=(40,46),
)
colors_q = palette(cols2,8)
list_q = []
list_norm_s_mean_sigma_end = []
#p = [-28.56060772780685, 31.065245277611137]
#@. fm(x,p) = p[1]+p[2]*x
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    push!(list_q,q)
    push!(list_norm_s_mean_sigma_end,list_norm_s_mean[end])
    #plot!(
    scatter!(
        list_sigma,
        list_norm_s_mean/100,
        #list_norm_s_mean/m^0.6/log(m),
        #list_norm_s_mean/fm(m,p),
        #yerr=list_rho_r_n_std/N,
        label="\$q=$q\$",
        c=colors_q[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
#savefig("figs/tmp.pdf")

B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
p = [-28.56060772780685, 31.065245277611137]
#@. fm(x,p) = p[1]+p[2]*x
@. fm(x,p) = p[1]+p[2]*x
fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_s_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    subplot=2,
    list_q,
    list_norm_s_mean_sigma_end/100,
    xlabel=L"q",
    ylabel=L"10^{-2}|s|(\sigma=5)",
    #xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    subplot=2,
    list_q[1]:1:list_q[end],
    x->fm(x,p)/100,
    #label=L"(-0.28\pm 0.01)+(0.311\pm 0.002)q",
    label="",
)
plot!(
    subplot=2,
    xticks=([2,4,6,8],[L"2",L"4",L"6",L"8"]),
    yticks=([0,1,2],[L"0",L"1",L"2"]),
    tickfont=font(11),
    #ylims=(40,46),
)
plot!()

In [ ]:
#p = [1.0]
#@. fm(x,p) = p[1]*x
p = [1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x

fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_s_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_q,
    list_norm_s_mean_sigma_end,
    xlabel=L"q",
    ylabel=L"|s|(\sigma=5)",
    #xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    list_q[1]:1:list_q[end],
    x->fm(x,p),
    label=L"(-28\pm 1)+(31.1\pm 0.2)q",
)

##### $|h|$ varying $N$

In [ ]:
figSM11c_1 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-0,1e2))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
    #ylims=(40,46),
)
plot!(xlabel=L"\sigma")
#plot!(ylabel=L"N^{-0.4}|h|/\ln N")
plot!(ylabel=L"10^{-2}|h|")
colors_N = palette(cols1,6)
list_N=[]
list_norm_h_mean_sigma_end=[]
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if q != 3
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_h_mean_sigma_end,list_norm_h_mean[end])
    #plot!(
    scatter!(
        list_sigma,
        list_norm_h_mean/100,
        #list_norm_h_mean/N^0.4/log(N),
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
#savefig("figs/tmp.pdf")

B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
p = [1.0,1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]
fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    subplot=2,
    list_N,
    list_norm_h_mean_sigma_end/100,
    xlabel=L"N",
    ylabel=L"10^{-2}|h|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    #ylim=(0,42),
    label="",
    #legend=:topleft,
    legend=:top,
)
plot!(
    subplot=2,
    list_N[1]:1:list_N[end],
    x->fm(x,p)/100,
    #label=L"(-0.075\pm 0.005)+(0.0232\pm 0.0003)N^{0.378\pm 0.002}\ln N",
    label="",
)
plot!(
    subplot=2,
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    #yticks=([0,1,2,3,4],[L"0",L"1",L"2",L"3",L"4"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
    ylims=(0,3.2),
)
plot!()

In [ ]:
p = [1.0,1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]

fit_fm = LsqFit.curve_fit(fm,list_N,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_N,
    list_norm_h_mean_sigma_end,
    xlabel=L"N",
    ylabel=L"|h|(\sigma=5)",
    #ylabel=L"10^{-1}|h|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    list_N[1]:1:list_N[end],
    x->fm(x,p),
    label=L"(-7.5\pm 0.5)+(2.32\pm 0.03)N^{0.378\pm 0.002}\ln N",
    #label=L"(-0.75\pm 0.05)+(0.232\pm 0.003)N^{0.378\pm 0.002}\ln N",
)

##### $|h|$ varying $q$

In [ ]:
figSM11c_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-0,1e2))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    tickfont=font(11),
    #ylims=(40,46),
)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10^{-2}|h|")
#colors_m_aux = vcat([c for c in colors_m[1:7]],[:red])
colors_q = palette(cols2,8)
list_q=[]
list_norm_h_mean_sigma_end=[]
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    push!(list_q,q)
    push!(list_norm_h_mean_sigma_end,list_norm_h_mean[end])
    #plot!(
    scatter!(
        list_sigma,
        list_norm_h_mean/100,#/N/m^0.59/log(m),
        #list_norm_h_mean/m^0.5*log(m),
        #yerr=list_rho_r_n_std/N,
        label="\$q=$q\$",
        c=colors_q[c],
        #c=colors_m_aux[c],
        markersize=3.0,
        markerstrokewidth=0.0,            
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
#savefig("figs/tmp.pdf")

B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
#p = [1.0,1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]*log(x)
fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
scatter!(
    subplot=2,
    list_q,
    list_norm_h_mean_sigma_end/100,
    xlabel=L"q",
    ylabel=L"10^{-2}|h|(\sigma=5)",
    #xscale=:log10,
    #yscale=:log10,
    label="",
    #ylim=(-1.0,30.0),
)
plot!(
    legend=:topleft,
    subplot=2,
    list_q[1]:0.1:list_q[end],
    x->fm(x,p)/100,
    #label=L"(192\pm 4)q^{-0.32\pm 0.01}\ln q",
    label="",
)
plot!(
    subplot=2,
    xticks=([2,4,6,8],[L"2",L"4",L"6",L"8"]),
    #yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    yticks=([0,1,2],[L"0",L"1",L"2"]),
    tickfont=font(11),
    #ylims=(-0.1,3),
    ylims=(-0.1,2.1),
)
plot!()

In [ ]:
#p = [1.0,1.0,1.0]
#@. fm(x,p) = p[1]+p[2]*x^p[3]*log(x)
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]*log(x)

fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_h_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_q,
    list_norm_h_mean_sigma_end,
    #list_norm_h_mean_sigma_end/10,
    xlabel=L"q",
    ylabel=L"|h|(\sigma=5)",
    #xscale=:log10,
    #yscale=:log10,
    label="",
    #ylim=(-1.0,23.0),
)
plot!(
    list_q[1]:0.1:list_q[end],
    x->fm(x,p),
    #x->fm(x,p)/10,
    label=L"(192\pm 4)q^{-0.32\pm 0.01}\ln q",
)

##### $|g|$ varying $N$

In [ ]:
figSM12c_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.65,0.75))
#plot!(yticks=0.65:0.05:0.75)
plot!(ylim=(0.5,1.0))
plot!(yticks=0.5:0.2:1.0)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([5,6,7,8,9],[L"5",L"6",L"7",L"8",L"9",]),
    tickfont=font(11),
    ylims=(5,9),
)
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if q != 3
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_g_mean/N^1.5*10,
        #list_norm_g_mean/N^1.35/log(N),
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",            
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
savefig("figs/tmp.pdf")
plot!()

##### $|g|$ varying $q$

In [ ]:
figSM12c_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.4,0.46))
#plot!(yticks=0.4:0.01:0.46)
plot!(ylim=(0.0,0.5))
plot!(yticks=0.0:0.1:0.5)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}q^{-1/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    #yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
    ylims=(3,5),
)
colors_q = palette(cols2,8)
list_q=[]
list_norm_g_mean_sigma0=[]
c=0
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    push!(list_q,q)
    #push!(list_norm_g_mean_sigma0,list_norm_g_mean[1])
    push!(list_norm_g_mean_sigma0,list_norm_g_mean[1]/N^1.5)
    #push!(list_norm_g_mean_sigma0,list_norm_g_mean[1]/N^1.5/m^0.5)
    scatter!(
        list_sigma,
        #list_norm_g_mean,#/N/m^0.59/log(m),
        list_norm_g_mean/N^1.5/q^0.5*10,#/log(m),
        #yerr=list_rho_r_n_std/N,
        label="\$q=$q\$",
        c=colors_q[c],
        markersize=3.0,
        markerstrokewidth=0.0, 
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]

fit_fm = LsqFit.curve_fit(fm,list_q,list_norm_g_mean_sigma0,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_q,
    list_norm_g_mean_sigma0,
    xlabel=L"q",
    ylabel=L"N^{-3/2}|g|(\sigma=0)",
    label="",
)
plot!(
    x->fm(x,p),
    label=L"(0.423\pm 0.001)q^{0.486\pm 0.001}",
    #ylim=(0.0,1.0),
)

##### $\kappa_k$ vs $N$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13c_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{BA},\;q=3",legendtitlefontsize=8)
plot!(
    xticks=([1e2,1e3],[L"10^2",L"10^3"]),
    yticks=([1e2,1e3,1e4],[L"10^2",L"10^3",L"10^4"]),
    tickfont=font(11),
)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"N")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))
list_N = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if q == 3
        push!(list_N,N)
        list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_N,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_N,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_N,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_N,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2
)

scatter!(
    list_N,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
@. fm2(x,p) = p[1]*x^p[2]
imin=1
fit_m2 = LsqFit.curve_fit(fm2,list_N[imin:end],list_m_2_mean[imin:end],[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_N[imin:end],
    #list_m_0_mean,
    x->fm2(x,fit_m2.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto N^{0.41\pm 0.02}",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3
)
@show R_squared(list_m_2_mean[imin:end],fm2(list_N[imin:end],fit_m2.param))

plot!()

##### $\kappa_k$ vs $q$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13c_2 = plot()
plot!(legend=:right,legendtitle=L"\mathrm{BA},\;N=512",legendtitlefontsize=8)
plot!(
    xticks=([1,2,4,8],[L"2^0",L"2^1",L"2^2",L"2^3"]),
    yticks=([1e0,1e1,1e2,1e3,1e4],[L"10^0",L"10^1",L"10^2",L"10^3",L"10^4"]),
    tickfont=font(11),
)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"q")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
#plot!(xlim=(1.0e-1,:auto))
plot!(ylim=(1.0e-4,:auto))
plot!(ylim=(1.0,1.0e4))
plot!(legend=:bottomright)
list_q = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(BA_stats_vs_sigma)))
    (N,q) = ky
    if N == 512
        push!(list_q,q)
        list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = BA_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_q,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_q,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_q,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_q,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_q,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_q,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto q^{0.991\pm 0.001}",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2
)

scatter!(
    list_q,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_q,list_m_2_mean,[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_q,
    #list_m_0_mean,
    x->fm2(x,fit_m2.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto q^{2.66\pm 0.02}",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3
)

plot!()

### Watts-Strogatz

#### Developing example

#### Small-world effect

In [ ]:
list_cc_mean[1]/list_cc_mean[1]

#### Disorder parameter vs noise

In [ ]:
filename = "data/WS_stats_vs_sigma.jld2"

# Deserialize data
if isfile(filename)
    @load filename WS_stats_vs_sigma
else    
    WS_stats_vs_sigma = Dict()    
end
sort(collect(keys(WS_stats_vs_sigma)))

##### $\bar{\tau}$ varying $N$

In [ ]:
fig2d_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\tau}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"\mathrm{WS},\;k=4, p=0.1",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\;\bar{k}=4, p=0.1",legendtitlefontsize=8)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p == 0.1
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_tau_mean/N,
            #yerr=list_N_tau_std/N,
            #label="\$N=$N\$  \$k=$k\$ \$p=$p\$"
            label="\$N=$N\$",
            c=colors_N[c],
            markersize=3.0,
            markerstrokewidth=0.0,                    
        )
    end
end    
plot!(0.0:0.1:5.0,x->0.92*x,label=L"0.92\sigma",c=:black,style=:dash,width=3.0)
plot!()

##### $\bar{\rho}$ varying $N$

In [ ]:
fig3d_1 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"k=4, p=0.1",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\;\bar{k}=4, p=0.1",legendtitlefontsize=8)

colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p == 0.1
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_rho_mean/N,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$k=$k\$ \$p=$p\$"
            label="\$N=$N\$",
            c=colors_N[c],
            markersize=3.0,
            markerstrokewidth=0.0,            
        )
    end
end    
plot!()

##### $\bar{\tau}$ varying $p$

In [ ]:
fig2d_2 = plot()
#plot!(yscale=:log10)
plot!(ylim=(-0.5,21.0))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\tau}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"N=512, k=4",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,5,10,15,20],[L"0",L"5",L"10",L"15",L"20"]),
    tickfont=font(11),
)
plot!(legend=:bottomright,legendtitle=L"\mathrm{WS},N=512, \bar{k}=4",legendtitlefontsize=8)
colors_p = palette(cols2,5)
colors_p_aux = Vector{Any}(undef,5)
colors_p_aux .= colors_p
colors_p_aux[4] = :red
list_p=[]
list_N_tau_mean_sigma_end=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N == 512
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]     
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        push!(list_p,p)
        push!(list_N_tau_mean_sigma_end,list_N_tau_mean[end]/N)
        scatter!(
            list_sigma,
            list_N_tau_mean/N,
            #yerr=list_N_tau_std/N,
            #label="\$N=$N\$  \$k=$k\$ \$p=$p\$",
            label="\$p=$p\$",
            #c=colors_p[c],
            c=colors_p_aux[c],
            markersize=3.0,
            markerstrokewidth=0.0,            
        )
    end
end    
B = bbox(0.22,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
scatter!(
    subplot=2,
    list_p,
    list_N_tau_mean_sigma_end,#/1000,
    xlabel=L"p",
    ylabel=L"\langle\bar{\tau}\rangle(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    xticks=[1e-4,1e-2,1e-0],
    label="",
    xlim=(5e-5,1.4),
    ylim=(-0.5,17.0),
)
plot!(
    subplot=2,
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^0"]),
    yticks=([0,5,10,15],[L"0",L"5",L"10",L"15"]),
    tickfont=font(11),
)
p = [20.0,5.0,0.001]
@. fm(x,p) = p[1]-p[2]/(1+p[3]/x)
fit_fm = LsqFit.curve_fit(fm,list_p,list_N_tau_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)
plot!(
    subplot=2,
    [10^y for y in -4.0:0.1:0.0],
    x->fm(x,p),
    #label=L"(15.9\pm 0.2)-(12.4\pm 0.3)/(1+(0.008\pm 0.0004)/p)",
    label="",
)
#plot!()

In [ ]:
#p = [1.0,20.0,0.1,0.001]
#@. fm(x,p) = p[1]+p[2]/(1+exp(p[3]*(exp(x)-p[4])))
#p = [1.0,1.0]
#@. fm(x,p) = p[1]*x^p[2]
#p = [20.0,5.0,1.0,0.001]
#@. fm(x,p) = p[1]-p[2]/(1+(p[4]/x)^p[3])
p = [20.0,5.0,0.001]
@. fm(x,p) = p[1]-p[2]/(1+p[3]/x)

fit_fm = LsqFit.curve_fit(fm,list_p,list_N_tau_mean_sigma_end,p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_p,
    list_N_tau_mean_sigma_end,
    xlabel=L"p",
    ylabel=L"\langle\bar{\tau}\rangle(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
    #ylim=(-1.0,23.0)
)
plot!(
    [10^y for y in -4.0:0.1:0.0],
    x->fm(x,p),
    label=L"(15.9\pm 0.2)-(12.4\pm 0.3)/(1+(0.008\pm 0.0004)/p)",
)

##### $\bar{\rho}$ varying $p$

In [ ]:
fig3d_2 = plot()
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"\langle\bar{\rho}\rangle")
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(0.0:0.0,x->0.0,label=L"N=512, k=4",width=0.0,c=:white)
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},N=512, \bar{k}=4",legendtitlefontsize=8)
colors_p_aux = Vector{Any}(undef,length(colors_p))
colors_p_aux .= colors_p
colors_p_aux[4] = :red

colors_p = palette(cols2,5)
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N == 512
        c+=1
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
        list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
        list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
        list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
        list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
        scatter!(
            list_sigma,
            list_N_rho_mean/N,
            #yerr=list_N_rho_std/N,
            #label="\$N=$N\$  \$k=$k\$ \$p=$p\$",
            label="\$p=$p\$",
            #c=colors_p[c],
            c=colors_p_aux[c],
            markersize=3.0,
            markerstrokewidth=0.0,            
        )
    end
end    
plot!()

##### $\sigma_c$

In [ ]:
WS_peaks = Dict()

for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,pp) = ky
    #if k != 1.0
    #    continue
    #end
    #@show ky
    
    list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_N_tau_mean = [mean(list_N_tau[t]) for t in 1:length(list_sigma)]
    list_N_tau_std  = [std(list_N_tau[t])  for t in 1:length(list_sigma)]
    list_N_rho_mean = [mean(list_N_rho[t]) for t in 1:length(list_sigma)]
    list_N_rho_std  = [std(list_N_rho[t])  for t in 1:length(list_sigma)]
   
    plt = plot()
    plot!(legend=:topright,legendtitle=L"\mathrm{WS}",legendtitlefontsize=8)
    #plot!(xlim=(0.0,2.0))
    #plot!(ylim=(-0.5,0.5))
    #plot!(ylim=(-0.5,:auto))
    plot!(xlabel=L"\sigma")
    plot!(ylabel=L"\langle\bar{\rho}\rangle")    

    r_star = find_first_nonzero(list_N_rho_mean)-1
    sigma_star = list_sigma[r_star]
    
    list_r = 60:200
    
    list_sigma_c_vs_r        = []
    list_sigma_c_err_vs_r    = []
    list_d2rho_peak_vs_r     = []
    list_d2rho_peak_err_vs_r = []
    list_mresid_vs_r         = []
    bestfit_r                = 0
    bestfit_list_sigma       = 0
    bestfit_p                = 0
    bestfit_sigma_c          = 0
    bestfit_sigma_c_err      = 0
    bestfit_d2rho_peak       = 0
    bestfit_d2rho_peak_err   = 0
    #p = [10.0,1.3,0.25,0.0]
    p = [1.3,0.25,10.0,0.0]
    for r in list_r
        p0 = p
        fit_range = r_star:r
        fit_list_sigma = collect(list_sigma[fit_range])
        fit_list_N_rho_mean = list_N_rho_mean[fit_range]
        fit = LsqFit.curve_fit(psigmoid,fit_list_sigma,fit_list_N_rho_mean/N,p0)   
        mresid = norm(fit.resid)/r # mean residuals
        p = fit.param
        #@show N,z,q,mresid,p
        p_err = LsqFit.standard_errors(fit)
        for i in 1:length(p)
            pi = p[i]
            pe = p_err[i]
        end
        #sigma_c          = p[2]
        #sigma_c_err      = p_err[2]
        #d2rho_peak       = dsigmoid(sigma_c,fit.param)
        #d2rho_peak_err   = estimate_error(dsigmoid,sigma_c,fit)
        A                = p[2]
        A_err            = p_err[2]
        B                = p[3]
        B_err            = p_err[3]
        sigma_c          = p[1]
        sigma_c_err      = p_err[1]
        d2rho_peak       = A*B/4 #dsigmoid(sigma_c,fit.param)
        d2rho_peak_err   = sqrt((B/4*A_err)^2+(A/4*B_err)^2) #estimate_error(dsigmoid,sigma_c,fit)                     
        push!(list_sigma_c_vs_r,sigma_c)
        push!(list_sigma_c_err_vs_r,sigma_c_err)
        push!(list_d2rho_peak_vs_r,d2rho_peak)
        push!(list_d2rho_peak_err_vs_r,d2rho_peak_err)
        push!(list_mresid_vs_r,mresid)
        if d2rho_peak >= bestfit_d2rho_peak
            bestfit_r              = r
            bestfit_list_sigma     = fit_list_sigma
            bestfit_p              = p
            bestfit_sigma_c        = sigma_c
            bestfit_sigma_c_err    = sigma_c_err
            bestfit_d2rho_peak     = d2rho_peak
            bestfit_d2rho_peak_err = d2rho_peak_err
        end
    end      
    @show N,pp,bestfit_r,bestfit_sigma_c,bestfit_d2rho_peak,list_r
    
    kk=Int(k)
    scatter!(
        list_sigma,
        list_N_rho_mean/N^1.0,
        label="\$N=$N\$, \$p=$pp\$, \$\\bar{k}=$(kk)\$",
        markersize=2.0,
        markerstrokewidth=0.0,        
    )
    plot!(bestfit_list_sigma,x->psigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle",width=2.0,color=:black,style=:dash)
    plot!(bestfit_list_sigma,x->sigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle'",width=2.0)
    plot!(bestfit_list_sigma,x->dsigmoid(x,bestfit_p),label=L"\langle\bar{\rho}\rangle''",width=2.0)
    #scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\bar{\rho}\rangle''(\sigma_c)",markersize=4.0,markerstrokewidth=0.0)
    scatter!([bestfit_sigma_c],[bestfit_d2rho_peak],label=L"\langle\tilde{\rho}\rangle",markersize=4.0,markerstrokewidth=0.0)
    scatter!([sigma_star],[0.0],label=L"\sigma^*",markersize=4.0,markerstrokewidth=0.0,c=:red,cl=:red,mc=:red)
    plot!(legend=:topright)
    
    #B = bbox(0.1,0.01,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    B = bbox(0.45,0.45,0.4,0.4) # The box is what allows the inset to have arbitrary position and dimensions.
    plot!(plt,inset = (1, B),subplot = 2)
    #plot!(xlabel=L"r",subplot=2)
    plot!(xlabel=L"\sigma^{**}",subplot=2)
    plot!(ylabel=L"\sigma_c",subplot=2,yguidefont = font(12,color=col6),)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],label="",subplot=2,c=col6,lc=col6,mc=col6)
    #plot!(list_r[1:10:end],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    plot!(list_sigma[list_r[1:10:end]],list_sigma_c_vs_r[1:10:end],yerr=list_sigma_c_err_vs_r[1:10:end],seriestype=:scatter,label="",subplot=2,c=col6,lc=col6,mc=col6,markersize=4.0,markerstrokewidth=0.0)
    #plot!(list_r,list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    plot!(list_sigma[list_r],list_sigma_c_vs_r,label="",subplot=2,c=col6,lc=col6,mc=col6)
    #scatter!([bestfit_r],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    scatter!([list_sigma[bestfit_r]],[bestfit_sigma_c],subplot=2,label="",c=:black,lc=:black,mc=:black)
    
    plt2 = twinx(plt[2])
    plot!(
        plt2,
        #xlabel=L"N",
        #ylabel=L"\langle\bar{\rho}\rangle''(\sigma_c)",
        ylabel=L"\langle\bar{\rho}\rangle",
        yguidefont = font(12,color=col7),
        #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
        #xscale=:log10,
        #yscale=:log10,
        subplot=2,
    )
    #scatter!(plt2,list_r[1:10:end],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    scatter!(plt2,list_sigma[list_r[1:10:end]],list_d2rho_peak_vs_r[1:10:end],yerr=list_d2rho_peak_err_vs_r[1:10:end],label="",subplot=2,c=col7,lc=col7,mc=col7,marker=:star,markersize=4.0,markerstrokewidth=0.0)
    #plot!(plt2,list_r,list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    plot!(plt2,list_sigma[list_r],list_d2rho_peak_vs_r,label="",subplot=2,c=col7,lc=col7,mc=col7)    
    
    WS_peaks[ky] = [plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star]    
    savefig("figs/SM-WS-sigma_c-N$N-p$pp.pdf")

end  
WS_peaks[(512,4,0.1)][1]

###### $\sigma_c$ vs $N$

In [ ]:
fig4d_1 = plot()
#plot!(xlim=(0.0,5.5))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
plot!(xscale=:log10)
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
plot!(
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([0.5,0.6,0.7],[L"0.5",L"0.6",L"0.7"]),
    tickfont=font(11),
)

list_N                   = []
list_sigma_c_vs_N        = []
list_sigma_c_err_vs_N    = []
list_d2rho_peak_vs_N     = []
list_d2rho_peak_err_vs_N = []
for ky in sort(collect(keys(WS_peaks)))
    (N,k,pp) = ky    
    if pp != 0.1
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = WS_peaks[ky]
    push!(list_N,N)
    push!(list_sigma_c_vs_N,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_N,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_N,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_N,bestfit_d2rho_peak_err)
end
@show list_N,list_sigma_c_vs_N,list_d2rho_peak_vs_N

plot!(ylims=(0.5,0.71))
plot!(xlabel=L"N")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_N,list_sigma_c_vs_N,yerr=list_sigma_c_err_vs_N,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_N,list_sigma_c_vs_N,label="",c=col6,lc=col6,mc=col6)

fig4d_1_t = twinx(fig4d_1)
plot!(
    fig4d_1_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    ylims=(1.1,1.41),
    #yscale=:log10,
)
plot!(fig4d_1_t,list_N,list_d2rho_peak_vs_N,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4d_1_t,list_N,list_d2rho_peak_vs_N,yerr=list_d2rho_peak_err_vs_N,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    #xticks=([2^6,2^7,2^8,2^9,2^10,2^11],[L"2^6",L"2^7",L"2^8",L"2^9",L"2^{10}",L"2^{11}"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([1.1,1.2,1.3,1.4],[L"1.1",L"1.2",L"1.3",L"1.4"]),
    tickfont=font(11),
)
#savefig("figs/tmp-ER-sigma_c-vs-N.pdf")
plot!()

###### $\sigma_c$ vs $p$

In [ ]:
fig4d_2 = plot()
plot!(ylim=(0.4,0.71))
#plot!(legend=:bottomleft)
#plot!(legend=:topleft)
#plot!(legend=:bottomright)
#plot!(xscale=:log10)
plot!(
    #xticks=([1e-4,1e-3,1e-2,1e-1,1],[L"10^{-4}",L"10^{-3}",L"10^{-2}",L"10^{-1}",L"10^{0}"]),
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{0}"]),
    yticks=([0.4,0.5,0.6,0.7],[L"0.4",L"0.5",L"0.6",L"0.7"]),
    tickfont=font(11),
)
plot!(legend=:left,legendtitle=L"\mathrm{WS},N=512,\bar{k}=4",legendtitlefontsize=8)

list_p                   = []
list_sigma_c_vs_p        = []
list_sigma_c_err_vs_p    = []
list_d2rho_peak_vs_p     = []
list_d2rho_peak_err_vs_p = []
for ky in sort(collect(keys(WS_peaks)))
    (N,k,pp) = ky    
    if N != 512
        continue
    end
    (plt,bestfit_sigma_c,bestfit_sigma_c_err,bestfit_d2rho_peak,bestfit_d2rho_peak_err,sigma_star) = WS_peaks[ky]
    push!(list_p,pp)
    push!(list_sigma_c_vs_p,bestfit_sigma_c)
    push!(list_sigma_c_err_vs_p,bestfit_sigma_c_err)
    push!(list_d2rho_peak_vs_p,bestfit_d2rho_peak)
    push!(list_d2rho_peak_err_vs_p,bestfit_d2rho_peak_err)
end
@show list_p,list_sigma_c_vs_p,list_d2rho_peak_vs_p

#plot!(xlims=(0,:auto))
plot!(xscale=:log10)
plot!(xlabel=L"p")
plot!(ylabel=L"\sigma_c",yguidefont = font(12,color=col6),)
plot!(list_p,list_sigma_c_vs_p,yerr=list_sigma_c_err_vs_p,seriestype=:scatter,label="",c=col6,lc=col6,mc=col6,markerstrokecolor=col6,markersize=4.0,markerstrokewidth=1.0)
plot!(list_p,list_sigma_c_vs_p,label="",c=col6,lc=col6,mc=col6)

fig4d_2_t = twinx(fig4d_2)
plot!(
    fig4d_2_t,
    #xlabel=L"N",
    ylabel=L"\langle\tilde{\rho}\rangle",
    yguidefont = font(12,color=col7),
    #ylabel=L"\textcolor{red}{x \, \text{axis label}}",
    xscale=:log10,
    #yscale=:log10,
    ylims=(1.0,2.11),
)
#plot!(   fig4d_2_t,yscale=:log10)  
plot!(   fig4d_2_t,list_p,list_d2rho_peak_vs_p,label="",c=col7,lc=col7,mc=col7)  
scatter!(fig4d_2_t,list_p,list_d2rho_peak_vs_p,yerr=list_d2rho_peak_err_vs_p,label="",c=col7,lc=col7,mc=col7,markerstrokecolor=col7,marker=:star,markersize=4.0,markerstrokewidth=1.0)
plot!(
    subplot=2,
    #xticks=([1e-4,1e-3,1e-2,1e-1,1],[L"10^{-4}",L"10^{-3}",L"10^{-2}",L"10^{-1}",L"10^{0}"]),
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{0}"]),
    yticks=([1.0,1.2,1.4,1.6,1.8,2.0],[L"1.0",L"1.2",L"1.4",L"1.6",L"1.8",L"2.0"]),
    tickfont=font(11),
)
#savefig("figs/tmp-ER-sigma_c-vs-k.pdf")
plot!()

##### $|f|$ varying $N$

In [ ]:
figSM9d_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.0,0.51))
#plot!(yticks=0.0:0.1:0.5)
plot!(ylim=(0,5.1))
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    tickfont=font(11),
)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}|f|")
colors_N = palette(cols1,6)
list_N=[]
list_norm_f_mean_sigma0 = []
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p != 0.1
       continue 
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_f_mean_sigma0,list_norm_f_mean[1])
    scatter!(
        list_sigma,
        list_norm_f_mean/N^1.5*10,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0, 
    )
    
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]

imin = 1

fit_fm = LsqFit.curve_fit(fm,list_N[imin:end],list_norm_f_mean_sigma0[imin:end],p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_N,
    list_norm_f_mean_sigma0,
    xscale=:log10,
    yscale=:log10,
    xlabel=L"N",
    ylabel=L"|f|",
    label="",
)
plot!(
    x->fm(x,p),
    label=L"(0.24\pm 0.01)N^{1.465\pm 0.007}",
)

##### $|f|$ varying $p$

In [ ]:
figSM9d_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{WS},N=512,\bar{k}=4",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.01))
plot!(ylim=(0.0,8))
#plot!(yticks=0.0:0.2:1.0)
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}|f|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,2,4,6,8],[L"0",L"2",L"4",L"6",L"8"]),    
    tickfont=font(11),
)
colors_p = palette(cols2,5)
list_p=[]
list_norm_f_mean_sigma0=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_f_mean = [mean(list_norm_f[t]) for t in 1:length(list_sigma)]
    list_norm_f_std  = [std(list_norm_f[t])  for t in 1:length(list_sigma)]
    push!(list_p,p)
    push!(list_norm_f_mean_sigma0,list_norm_f_mean[1]/N^1.5)
    scatter!(
        list_sigma,
        list_norm_f_mean/N^1.5*10,
        #yerr=list_rho_r_n_std/N,
        label="\$p=$p\$",
        c=colors_p[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
scatter!(
    subplot=2,
    list_p,
    list_norm_f_mean_sigma0*10,
    xlabel=L"p",
    ylabel=L"10 N^{-3/2}|f|(\sigma=0)",
    xscale=:log10,
    #yscale=:log10,
    label="",
    #xlim=(5e-5,1.2),
    #ylim=(-0.5,17.0),
    #yticks=[]
)
plot!(
    subplot=2,
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^0"]),
    yticks=([0,2,4,6,8],[L"0",L"2",L"4",L"6",L"8"]),    
    ylim=(0,6),
    tickfont=font(11),
)
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
plot()
scatter!(
    list_p,
    list_norm_f_mean_sigma0,
    xscale=:log10,
    #yscale=:log10,
    xlabel=L"p",
    ylabel=L"N^{-3/2}|f|(\sigma=0)",
    label="",
    legend=:topleft,
)

##### $|s|$ varying $N$

In [ ]:
figSM10d_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|s|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
)
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p != 0.1
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_s_mean/N^0.5,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

##### $|s|$ varying $p$

In [ ]:
figSM10d_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{WS},\bar{k}=4,N=512",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-3,1e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|s|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
)
colors_p = palette(cols2,5)
list_p=[]
list_norm_s_mean_sigma_end=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_s_mean = [mean(list_norm_s[t]) for t in 1:length(list_sigma)]
    list_norm_s_std  = [std(list_norm_s[t])  for t in 1:length(list_sigma)]
    push!(list_p,p)
    push!(list_norm_s_mean_sigma_end,list_norm_s_mean[end]/N^0.5)
    scatter!(
        list_sigma,
        list_norm_s_mean/N^0.5,
        #yerr=list_rho_r_n_std/N,
        label="\$p=$p\$",
        c=colors_p[c],
        markersize=3.0,
        markerstrokewidth=0.0,           
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
scatter!(
    subplot=2,
    list_p,
    list_norm_s_mean_sigma_end,
    xlabel=L"p",
    ylabel=L"N^{-1/2}|s|(\sigma=5)",
    xscale=:log10,
    label="",
    ylim=(0.0,5.2),
)
plot!(
    subplot=2,
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{0}"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
)

In [ ]:
plot()
scatter!(
    list_p,
    list_norm_s_mean_sigma_end,
    xlabel=L"p",
    ylabel=L"N^{-1/2}|s|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
    ylim=(0.1,5.1),
)

##### $|h|$ varying $N$

In [ ]:
figSM11d_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-1,1e1))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|h|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
    ylim=(0,3),
)
colors_N = palette(cols1,6)
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p != 0.1
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    scatter!(
        list_sigma,
        list_norm_h_mean/N^0.5,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,          
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

##### $|h|$ varying $p$

In [ ]:
figSM11d_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{WS},N=512,\bar{k}=4",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1e-3,1e3))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"N^{-1/2}|h|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
)
list_p=[]
list_norm_h_mean_sigma_end=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N != 512
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_h_mean = [mean(list_norm_h[t]) for t in 1:length(list_sigma)]
    list_norm_h_std  = [std(list_norm_h[t])  for t in 1:length(list_sigma)]
    push!(list_p,p)
    push!(list_norm_h_mean_sigma_end,list_norm_h_mean[end]/N^0.5)
    scatter!(
        list_sigma,
        list_norm_h_mean/N^0.5,
        #yerr=list_rho_r_n_std/N,
        label="\$p=$p\$",
        c=colors_p[c],
        markersize=3.0,
        markerstrokewidth=0.0,              
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
#B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
B = bbox(0.45,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
scatter!(
    subplot=2,
    list_p,
    list_norm_h_mean_sigma_end,
    xlabel=L"p",
    ylabel=L"N^{-1/2}|h|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    subplot=2,
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{-0}"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
    ylim=(0,5.1),
)

In [ ]:
imin = 1
p = [0.0,1.0,1.0,1.0]
@. fm(x,p) = p[1]+p[2]*x^p[3]*log(p[4]+x)

@show list_p[imin:end]
@show list_norm_h_mean_sigma_end[imin:end]

fit_fm = LsqFit.curve_fit(fm,list_p[imin:end],list_norm_h_mean_sigma_end[imin:end],p)
@show p = fit_fm.param
#@show standard_errors(fit_fm)

plot()
scatter!(
    list_p,
    list_norm_h_mean_sigma_end,
    xlabel=L"p",
    ylabel=L"N^{-1/2}|h|(\sigma=5)",
    xscale=:log10,
    #yscale=:log10,
    label="",
)
plot!(
    logrange(list_p[imin],1.01,list_p[end]),
    x->fm(x,p),
    label="",
)

##### $|g|$ varying $N$

In [ ]:
figSM12d_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.0,0.5))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10N^{-3/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    yticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),    
    tickfont=font(11),
    ylim=(0,5)
)
colors_N = palette(cols1,6)
list_N=[]
list_norm_g_mean_sigma0=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p != 0.1
        continue
    end
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    push!(list_N,N)
    push!(list_norm_g_mean_sigma0,list_norm_g_mean[1])
    scatter!(
        list_sigma,
        list_norm_g_mean/N^1.5*10,
        #yerr=list_rho_r_n_std/N,
        label="\$N=$N\$",
        c=colors_N[c],
        markersize=3.0,
        markerstrokewidth=0.0,               
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
plot!()

In [ ]:
p = [1.0,1.0]
@. fm(x,p) = p[1]*x^p[2]
imin = 1
fit_fm = LsqFit.curve_fit(fm,list_N[imin:end],list_norm_g_mean_sigma0[imin:end],p)
@show p = fit_fm.param
@show standard_errors(fit_fm)

plot()
scatter!(
    list_N,
    list_norm_g_mean_sigma0,
    xlabel=L"N",
    ylabel=L"|g|",
    xscale=:log10,
    yscale=:log10,
    label="",
)
plot!(
    x->fm(x,p),
    label=L"(0.24\pm 0.01)N^{1.465\om 0.007}",
)

##### $|g|$ varying $p$

In [ ]:
figSM12d_2 = plot()
plot!(legend=:bottomright,legendtitle=L"\mathrm{WS},N=512,\bar{k}=4",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(0.0,0.5))
plot!(xlabel=L"\sigma")
plot!(ylabel=L"10 N^{-3/2}|g|")
plot!(
    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
    #yticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),    
    yticks=([0,2,4,6,8,10],[L"0",L"2",L"4",L"6",L"8",L"10"]),    
    tickfont=font(11),
    ylim=(0,10),
)
colors_p = palette(cols2,5)
list_p=[]
list_norm_g_mean_sigma0=[]
c=0
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if N != 512
        continue
    end
    println(N," ",10/N^1.5)
    c+=1
    list_sigma,list_rho_w_n,list_rho_r_n,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
    list_norm_g_mean = [mean(list_norm_g[t]) for t in 1:length(list_sigma)]
    list_norm_g_std  = [std(list_norm_g[t])  for t in 1:length(list_sigma)]
    push!(list_p,p)
    push!(list_norm_g_mean_sigma0,list_norm_g_mean[1]/N^1.5)
    scatter!(
        list_sigma,
        list_norm_g_mean/N^1.5*10,
        #yerr=list_rho_r_n_std/N,
        label="\$p=$p\$",
        c=colors_p[c],
        markersize=3.0,
        markerstrokewidth=0.0,         
    )
end  
#scatter!(list_sigma,list_rho_r_n_mean/N,yerr=list_rho_r_n_std/N,label="")
B = bbox(0.23,0.01,0.45,0.45) # The box is what allows the inset to have arbitrary position and dimensions.
plot!(inset=(1,B),subplot=2)
scatter!(
    subplot=2,
    list_p,
    list_norm_g_mean_sigma0*10,
    xlabel=L"p",
    ylabel=L"10 N^{-3/2}|g|(\sigma=0)",
    label="",
    xscale=:log10,
)
plot!(
    subplot=2,
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{0}"]),
    yticks=([0,1,2,3,4,5,6],[L"0",L"1",L"2",L"3",L"4",L"5",L"6"]),    
    tickfont=font(11),
    ylim=(0.0,6.1),
)

In [ ]:
# Here we compare the curves in the insets of Figs. SM9d, SM10d and SM12d.
plot()
scatter!(
    list_p,
    list_norm_g_mean_sigma0,
    xlabel=L"p",
    #ylabel=L"N^{-3/2}|g|(\sigma=0)",
    label=L"|g|",
    xscale=:log10,
    #yscale=:log10,
    markersize=4.0    
)
scatter!(
    list_p,
    list_norm_h_mean_sigma_end/10.0,
    xscale=:log10,
    #yscale=:log10,
    #xlabel=L"p",
    #ylabel=L"N^{-3/2}|f|(\sigma=0)",
    label=L"|h|",
    legend=:topleft,
    markersize=3.0
)
scatter!(
    list_p,
    list_norm_f_mean_sigma0,
    xscale=:log10,
    #yscale=:log10,
    #xlabel=L"p",
    #ylabel=L"N^{-3/2}|f|(\sigma=0)",
    label=L"|f|",
    legend=:topleft,
    markersize=2.0,
    xlims=(5*1e-5,1.1),
    ylims=(0,1)
)

##### $\kappa_k$ varying $N$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13d_1 = plot()
plot!(legend=:topleft,legendtitle=L"\mathrm{WS},\bar{k}=4,p=0.1",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"N")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(
    #xticks=([1e2,1e3],[L"10^2",L"10^3"]),
    xticks=([2^6,2^8,2^10],[L"2^6",L"2^8",L"2^{10}"]),
    yticks=([1e2,1e3],[L"10^2",L"10^3"]),    
    tickfont=font(11),
    #ylim=(0,5)
)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))

list_N = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,p) = ky
    if p == 0.1
        push!(list_N,N)
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_N,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_N,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_N,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_N,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2
)

scatter!(
    list_N,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
@. fm2(x,p) = p[1]*x^p[2]
fit_m2 = LsqFit.curve_fit(fm2,list_N,list_m_2_mean,[1.0,1.0])
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    list_N,
    #list_m_0_mean,
    x->fm2(x,fit_m2.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto N",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3
)

plot!()

##### $\kappa_k$ varying $p$

In [ ]:
# m_k = kappa_k to simplify notation
figSM13d_2 = plot()
plot!(legend=:bottomleft,legendtitle=L"\mathrm{WS},N=512,\bar{k}=4",legendtitlefontsize=8)
#plot!(yscale=:log10)
#plot!(ylim=(1.0e-5,1.0e3))
plot!(xlabel=L"p")
plot!(ylabel=L"\langle \kappa_k\rangle")
plot!(xscale=:log10)
plot!(yscale=:log10)
plot!(
    #xticks=([1e-4,1e-3,1e-2,1e-1,1],[L"10^{-4}",L"10^{-3}",L"10^{-2}",L"10^{-1}",L"10^{0}"]),    
    xticks=([1e-4,1e-2,1],[L"10^{-4}",L"10^{-2}",L"10^{0}"]),    
    yticks=([1e1,1e2,1e3],[L"10^1",L"10^2",L"10^3"]),    
    tickfont=font(11),
    #ylim=(0,5)
)
#plot!(xlim=(1.0e-1,:auto))
#plot!(ylim=(1.0e-4,:auto))

list_p = []
list_m_0_mean = []
list_m_0_std  = []
list_m_1_mean = []
list_m_1_std  = []
list_m_2_mean = []
list_m_2_std  = []
for ky in sort(collect(keys(WS_stats_vs_sigma)))
    (N,k,pp) = ky
    if N == 512
        push!(list_p,pp)
        list_sigma,list_N_tau,list_N_rho,list_norm_f,list_norm_s,list_norm_h,list_norm_g,m_0,m_1,m_2 = WS_stats_vs_sigma[ky]
        push!(list_m_0_mean,mean(m_0))
        push!(list_m_0_std ,std(m_0))
        push!(list_m_1_mean,mean(m_1))
        push!(list_m_1_std ,std(m_1))
        push!(list_m_2_mean,mean(m_2))
        push!(list_m_2_std ,std(m_2))
    end
end       
@show list_m_0_mean
@show list_m_1_mean
@show list_m_2_mean

scatter!(
    list_p,
    list_m_0_mean,
    #yerr=list_m_0_std,
    #label="\$m_0\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=1
)
@. fm0(x,p) = p[1]*x^p[2]
fit_m0 = LsqFit.curve_fit(fm0,list_p,list_m_0_mean,[1.0,1.0])
@show fit_m0.param
@show standard_errors(fit_m0)
plot!(
    list_p,
    #list_m_0_mean,
    x->fm0(x,fit_m0.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_0\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=1
)

scatter!(
    list_p,
    list_m_1_mean,
    #yerr=list_m_1_std,
    #label="\$m_1\$",
    label="",
    #xlims=(0,2),c
    #ylims=(1.0e-5,:auto),
    c=2
)
@. fm1(x,p) = p[1]*x^p[2]
fit_m1 = LsqFit.curve_fit(fm1,list_p,list_m_1_mean,[1.0,1.0])
@show fit_m1.param
@show standard_errors(fit_m1)
plot!(
    list_p,
    #list_m_0_mean,
    x->fm1(x,fit_m1.param),
    #yerr=list_m_0_std,
    label=L"\langle \kappa_1\rangle\propto 1",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=2
)

scatter!(
    list_p,
    list_m_2_mean,
    #yerr=list_m_2_std,
    #label="\$m_2\$",
    label="",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    c=3
)
#@. fm2(x,p) = p[1]*x^(-p[2])*exp(-x/p[3])
@. fm2(x,p) = p[1]*exp(-x/p[2])
p0 = [500.0,0.25]
fit_m2 = LsqFit.curve_fit(fm2,list_p,list_m_2_mean,p0)
@show fit_m2.param
@show standard_errors(fit_m2)
plot!(
    #list_p,
    logrange(list_p[1],1.2,list_p[end]),
    #list_m_0_mean,
    x->fm2(x,fit_m2.param),
    #x->fm2(x,p0),    
    #yerr=list_m_0_std,
    label=L"\langle \kappa_2\rangle\propto e^{-p/(0.30\pm 0.02)}",
    #xlims=(0,2),
    #ylims=(1.0e-5,:auto),
    style=:dash,
    width=1.0,
    c=3
)

plot!()

# Joining subplots

## Fig. 2

In [ ]:
fig2a = plot(fig2a_1,fig2a_2)
savefig("figs/fig2a.pdf")
plot(fig2a)

In [ ]:
fig2b = plot(fig2b_1,fig2b_2)
savefig("figs/fig2b.pdf")
plot(fig2b)

In [ ]:
fig2c = plot(fig2c_1,fig2c_2)
savefig("figs/fig2c.pdf")
plot(fig2c)

In [ ]:
fig2d = plot(fig2d_1,fig2d_2)
savefig("figs/fig2d.pdf")
plot(fig2d)

## Fig. 3

In [ ]:
fig3a = plot(fig3a_1,fig3a_2)
savefig("figs/fig3a.pdf")
plot(fig3a)

In [ ]:
fig3b = plot(fig3b_1,fig3b_2)
savefig("figs/fig3b.pdf")
plot(fig3b)

In [ ]:
fig3c = plot(fig3c_1,fig3c_2)
savefig("figs/fig3c.pdf")
plot(fig3c)

In [ ]:
fig3d = plot(fig3d_1,fig3d_2)
savefig("figs/fig3d.pdf")
plot(fig3d)

## Fig. 4

In [ ]:
fig4a = plot(fig4a_1,fig4a_2)
savefig("figs/fig4a.pdf")
plot(fig4a)

In [ ]:
fig4b = plot(fig4b_1,fig4b_2)
savefig("figs/fig4b.pdf")
plot(fig4b)

In [ ]:
fig4c = plot(fig4c_1,fig4c_2)
savefig("figs/fig4c.pdf")
plot(fig4c)

In [ ]:
fig4d = plot(fig4d_1,fig4d_2)
savefig("figs/fig4d.pdf")
plot(fig4d)

## Fig. SM9

In [ ]:
figSM9a = plot(figSM9a_1,figSM9a_2)
savefig("figs/figSM9a.pdf")
plot(figSM9a)

In [ ]:
figSM9b = plot(figSM9b_1,figSM9b_2)
savefig("figs/figSM9b.pdf")
plot(figSM9b)

In [ ]:
figSM9c = plot(figSM9c_1,figSM9c_2)
savefig("figs/figSM9c.pdf")
plot(figSM9c)

In [ ]:
figSM9d = plot(figSM9d_1,figSM9d_2)
savefig("figs/figSM9d.pdf")
plot(figSM9d)

## Fig. SM10

In [ ]:
figSM10a = plot(figSM10a_1,figSM10a_2)
savefig("figs/figSM10a.pdf")
plot(figSM10a)

In [ ]:
figSM10b = plot(figSM10b_1,figSM10b_2)
savefig("figs/figSM10b.pdf")
plot(figSM10b)

In [ ]:
figSM10c = plot(figSM10c_1,figSM10c_2)
savefig("figs/figSM10c.pdf")
plot(figSM10c)

In [ ]:
figSM10d = plot(figSM10d_1,figSM10d_2)
savefig("figs/figSM10d.pdf")
plot(figSM10d)

## Fig. SM11

In [ ]:
figSM11a = plot(figSM11a_1,figSM11a_2)
savefig("figs/figSM11a.pdf")
plot(figSM11a)

In [ ]:
figSM11b = plot(figSM11b_1,figSM11b_2)
savefig("figs/figSM11b.pdf")
plot(figSM11b)

In [ ]:
figSM11c = plot(figSM11c_1,figSM11c_2)
savefig("figs/figSM11c.pdf")
plot(figSM11c)

In [ ]:
figSM11d = plot(figSM11d_1,figSM11d_2)
savefig("figs/figSM11d.pdf")
plot(figSM11d)

## Fig. SM12

In [ ]:
figSM12a = plot(figSM12a_1,figSM12a_2)
savefig("figs/figSM12a.pdf")
plot(figSM12a)

In [ ]:
figSM12b = plot(figSM12b_1,figSM12b_2)
savefig("figs/figSM12b.pdf")
plot(figSM12b)

In [ ]:
figSM12c = plot(figSM12c_1,figSM12c_2)
savefig("figs/figSM12c.pdf")
plot(figSM12c)

In [ ]:
figSM12d = plot(figSM12d_1,figSM12d_2)
savefig("figs/figSM12d.pdf")
plot(figSM12d)

## Fig. SM13

In [ ]:
figSM13a = plot(figSM13a_1,figSM13a_2)
savefig("figs/figSM13a.pdf")
plot(figSM13a)

In [ ]:
figSM13b = plot(figSM13b_1,figSM13b_2)
savefig("figs/figSM13b.pdf")
plot(figSM13b)

In [ ]:
figSM13c = plot(figSM13c_1,figSM13c_2)
savefig("figs/figSM13c.pdf")
plot(figSM13c)

In [ ]:
figSM13d = plot(figSM13d_1,figSM13d_2)
savefig("figs/figSM13d.pdf")
plot(figSM13d)

# Exponentes

In [ ]:
imax=15
vals_r = zeros(imax*imax)
vals_i = zeros(Int64,imax*imax)
vals_j = zeros(Int64,imax*imax)
k = 0
for i in 1:imax
    for j in 1:imax
        k += 1
        r = i/j
        #println("i=$i j=$j i/j=$r")
        #println("$r $i $j")
        vals_i[k] = i
        vals_j[k] = j
        vals_r[k] = r
    end
end
sorted_k = sortperm(vals_r)
for k in sorted_k
    i = vals_i[k]
    j = vals_j[k]
    r = vals_r[k]
    println("$r $i $j")
end

# References

[1] [G. Bianconi, Higher-Order Networks, Elements in the Structure and Dynamics of Complex Networks (Cambridge University Press, 2021)](https://www.cambridge.org/core/books/higherorder-networks/26EE7ED4ECB114B2E231987C22759CDF)

[2] [J. L. Johnson and T. Goldring, Discrete hodge theory on graphs: A tutorial, Computing in Science & Engineering
15, 42 (2013)](https://doi.ieeecomputersociety.org/10.1109/MCSE.2012.91)

[3] [A. N. Hirani, K. Kalyanaraman and S. Watts, "Graph Laplacians and Least Squares on Graphs," 2015 IEEE International Parallel and Distributed Processing Symposium Workshop, Hyderabad, India, 2015, pp. 812-821, doi: 10.1109/IPDPSW.2015.73](https://ieeexplore.ieee.org/document/7284395)

[4] [A tutorial about **Numerical Computing in Julia** by MatecDev](https://www.matecdev.com/posts/julia-numerical-computing.html)

[5] [`IterativeSolvers.jl`: A Julia package that provides iterative algorithms for solving linear systems, eigensystems, and singular value problems.](https://juliahub.com/ui/Packages/General/IterativeSolvers)

[6] [`Krylov.jl`: A Julia basket of hand-picked Krylov methods](https://juliahub.com/ui/Packages/General/Krylov)

[7] [`KrylovKit.jl`: A Julia package collecting a number of Krylov-based algorithms for linear problems, singular value and eigenvalue problems and the application of functions of linear maps or operators to vectors](https://juliahub.com/ui/Packages/General/KrylovKit)

[8] [`PyDEC`: A Python Library for Discretizations of Exterior Calculus.](https://github.com/hirani/pydec/tree/master)

[9] X. Jiang, L.-H. Lim, Y. Yao, and Y. Ye, Statistical ranking and combinatorial hodge theory, Mathematical Programming 127, 203 (2011)

[10] S.-C. Choi, Iterative methods for singular linear equations and least-squares problems, Ph.D. thesis, Stanford University, Stanford, CA, 2006. [http://www.stanford.edu/group/
SOL/dissertations/sou-cheng-choi-thesis.pdf](http://www.stanford.edu/group/SOL/dissertations/sou-cheng-choi-thesis.pdf)

[11] S.-C. Choi and M.A. Saunders, “minres-qlp: a Krylov subspace method for singular symmetric systems and least squares problems,” preprint, (2008).

[12] [https://web.stanford.edu/group/SOL/software/minresqlp/](https://web.stanford.edu/group/SOL/software/minresqlp/)

[13] [https://discourse.julialang.org/t/solving-sparse-singular-linear-systems/33401](https://discourse.julialang.org/t/solving-sparse-singular-linear-systems/33401)

[14] [https://discourse.julialang.org/t/finding-the-least-norm-solution-of-least-squares-problem-with-a-sparse-matrix/30702](https://discourse.julialang.org/t/finding-the-least-norm-solution-of-least-squares-problem-with-a-sparse-matrix/30702)

[15] [https://jso.dev/Krylov.jl/dev/solvers/sid/#MINRES-QLP](https://jso.dev/Krylov.jl/dev/solvers/sid/#MINRES-QLP)

[16] [https://jso.dev/Krylov.jl/dev/solvers/ls/#LSQR](https://jso.dev/Krylov.jl/dev/solvers/ls/#LSQR)